# <img src="2stage_model.png">


## 2 Stage model test  
This .ipynb is test for 2 stage model  

#### 1. Load yolo output
- Input : yolo format txt results (class xmin ymin xmax ymax)
- Description : 
    - load yolo txt information and interpretation
    - classify bmo and lc and load images
    - make array and padding for fitted segmentor input
- Output : padded yolo results array, padding information

#### 2. Inference fine segmentor
- Input : padded yolo results array
- Description : 
    - overlap-tile inference
    - reconstruction images
- Output : Inferenced bmo, lc patch coordinate

#### 3. Reconstruction on test images
- Input : Inferenced bmo, lc patch coordinate, padding information
- Description : reconstruction images using coordinate info and padding info
- Output : Save reconstrcted images 



#### 1. Load yolo output
- Input : yolo format txt results (class xmin ymin xmax ymax)
- Description : 
    - load yolo txt information and interpretation
    - classify bmo and lc and load images
    - make array and padding for fitted segmentor input
- Output : padded yolo results array, padding information

In [18]:
import warnings
import sys

#if not sys.warnoptions:
#    warnings.simplefilter("ignore")
    
import os 
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import configparser

from keras.models import model_from_json



sys.path.insert(0, './lib_keras/')
from help_functions import *

VALID_TEST = 'valid'


In [19]:
def load_yolo_imgs(yolo_img_path,num_imgs,shape_imgs):
    img_h, img_w = shape_imgs[0], shape_imgs[1]
    yolo_imgs = np.zeros((num_imgs, img_h, img_w,3),dtype=np.uint8)    
    idx =0 
    
    for count, filename in enumerate(sorted(os.listdir(yolo_img_path)), start=0):
        if filename.startswith(".ipynb") == False:
            temp = cv2.imread(yolo_img_path + filename)
            yolo_imgs[idx] = temp.astype(np.uint8)
            idx = idx+1
            
    return yolo_imgs  
    
def load_and_classify_yolo_txt(yolo_txt_path):
    yolo_table = pd.DataFrame(columns=['lc_xmin', 'lc_ymin', 'lc_xmax', 'lc_ymax',
                                       'bmo1_xmin', 'bmo1_ymin', 'bmo1_xmax', 'bmo1_ymax',
                                      'bmo2_xmin', 'bmo2_ymin', 'bmo2_xmax', 'bmo2_ymax'])

    for count, filename in enumerate(sorted(os.listdir(yolo_txt_path)), start=0):
        if filename.startswith(".ipynb") == False:
            txt_data = pd.read_csv(yolo_txt_path + filename, sep="\n", header=None)
            txt_info_01 = (txt_data.loc[0])[0].split(' ')
            txt_info_02 = (txt_data.loc[1])[0].split(' ')
            txt_info_03 = (txt_data.loc[2])[0].split(' ')
            
            yolo_table.loc[count, 'lc_xmin'] = int(txt_info_01[2])
            yolo_table.loc[count, 'lc_ymin'] = int(txt_info_01[3])
            yolo_table.loc[count, 'lc_xmax'] = int(txt_info_01[4])
            yolo_table.loc[count, 'lc_ymax'] = int(txt_info_01[5])
            
            yolo_table.loc[count, 'bmo1_xmin'] = int(txt_info_02[2])
            yolo_table.loc[count, 'bmo1_ymin'] = int(txt_info_02[3])
            yolo_table.loc[count, 'bmo1_xmax'] = int(txt_info_02[4])
            yolo_table.loc[count, 'bmo1_ymax'] = int(txt_info_02[5])
            
            yolo_table.loc[count, 'bmo2_xmin'] = int(txt_info_03[2])
            yolo_table.loc[count, 'bmo2_ymin'] = int(txt_info_03[3])
            yolo_table.loc[count, 'bmo2_xmax'] = int(txt_info_03[4])
            yolo_table.loc[count, 'bmo2_ymax'] = int(txt_info_03[5])
                                  
    return yolo_table

def count_num_files(FILE_PATH):
    num_files = 0
    for count, filename in enumerate(sorted(os.listdir(FILE_PATH)), start=0):
        if filename.startswith(".ipynb") == False:
            num_files = num_files+1
    return num_files

def get_shape_of_imgs(SEG_PATH):
    for count, filename in enumerate(sorted(os.listdir(SEG_PATH)), start=0):
        if filename.startswith(".ipynb") == False:
            temp_imgs = cv2.imread(SEG_PATH + filename)
            return np.shape(temp_imgs)
        

In [20]:
# Hyperparam, constant

if VALID_TEST == 'valid':
    YOLO_IMG_PATH = '/home/bono/Desktop/mellab_project/bono_hw/keras-yolo3/test/pred_fine_test/'
    YOLO_TXT_PATH = '/home/bono/Desktop/mellab_project/bono_hw/keras-yolo3/test/pred_fine_testTxt/'
    ORI_IMG_PATH = '/home/bono/Desktop/mellab_project/bono_hw/oct_segmentation/data/fine_segmentor_data/fine_test/'
    GROUND_TRUTH_PATH = '/home/bono/Desktop/mellab_project/bono_hw/oct_segmentation/data/fine_segmentor_data/fine_test_label/'
    INFERENCE_SHAPE  = (64,64)

    '''
    Clinical test
    '''
    #YOLO_IMG_PATH = '/home/bono/Desktop/mellab_project/bono_hw/keras-yolo3/test/clinical_data/ambi_test_0709/'
    #YOLO_TXT_PATH = '/home/bono/Desktop/mellab_project/bono_hw/keras-yolo3/test/clinical_data/ambi_test_txt_0709/'
    #ORI_IMG_PATH = '/home/bono/Desktop/mellab_project/bono_hw/oct_segmentation/data/fine_segmentor_data/Final_clinical/ambiguos/'
    #GROUND_TRUTH_PATH = '/home/bono/Desktop/mellab_project/bono_hw/oct_segmentation/data/fine_segmentor_data/Final_clinical/ambiguos_label/'

    #YOLO_IMG_PATH = '/home/bono/Desktop/mellab_project/bono_hw/keras-yolo3/test/clinical_data/test_0709/'
    #YOLO_TXT_PATH = '/home/bono/Desktop/mellab_project/bono_hw/keras-yolo3/test/clinical_data/test_txt_0709/'
    #ORI_IMG_PATH = '/home/bono/Desktop/mellab_project/bono_hw/oct_segmentation/data/fine_segmentor_data/Final_clinical/final_fine_clinical_test/'
    #GROUND_TRUTH_PATH = '/home/bono/Desktop/mellab_project/bono_hw/oct_segmentation/data/fine_segmentor_data/Final_clinical/final_fine_clinical_label/'

    #latest
    #YOLO_IMG_PATH = '/home/bono/Desktop/mellab_project/bono_hw/keras-yolo3/test/clinical_data/best_0709/'
    #YOLO_TXT_PATH = '/home/bono/Desktop/mellab_project/bono_hw/keras-yolo3/test/clinical_data/best_txt_0709/'
    #ORI_IMG_PATH = '/home/bono/Desktop/mellab_project/bono_hw/oct_segmentation/data/fine_segmentor_data/Final_clinical/best_0709/'
    #GROUND_TRUTH_PATH = '/home/bono/Desktop/mellab_project/bono_hw/oct_segmentation/data/fine_segmentor_data/Final_clinical/best_label_0709/'

    #YOLO_IMG_PATH = '/home/bono/Desktop/mellab_project/bono_hw/keras-yolo3/test/clinical_data/test/'
    #YOLO_TXT_PATH = '/home/bono/Desktop/mellab_project/bono_hw/keras-yolo3/test/clinical_data/test_txt/'
    #ORI_IMG_PATH = '/home/bono/Desktop/mellab_project/bono_hw/oct_segmentation/data/fine_segmentor_data/fine_clinical_test/'
    #GROUND_TRUTH_PATH = '/home/bono/Desktop/mellab_project/bono_hw/oct_segmentation/data/fine_segmentor_data/fine_clinical_test_label/'

    # processing
    NUM_YOLO_RESULT = count_num_files(YOLO_IMG_PATH)
    SHAPE_YOLO_RESULT = get_shape_of_imgs(YOLO_IMG_PATH)
    print(SHAPE_YOLO_RESULT)
    YOLO_IMGS = load_yolo_imgs(YOLO_IMG_PATH, NUM_YOLO_RESULT, SHAPE_YOLO_RESULT) 
    ORI_IMGS = load_yolo_imgs(ORI_IMG_PATH, NUM_YOLO_RESULT, SHAPE_YOLO_RESULT)

    YOLO_RESULT_TABLE = load_and_classify_yolo_txt(YOLO_TXT_PATH)
    BMO_GROUND_TRUTH_IMGS = load_yolo_imgs(GROUND_TRUTH_PATH, NUM_YOLO_RESULT,SHAPE_YOLO_RESULT )

    # Result image (BMO + LC, 3ch)
    RESULT_IMGS = ORI_IMGS.copy()
    RESULT_IMGS_MASKS = np.zeros((NUM_YOLO_RESULT, SHAPE_YOLO_RESULT[0], SHAPE_YOLO_RESULT[1], SHAPE_YOLO_RESULT[2]), dtype=np.uint16)

    # BMO -> Center points (255,0,0)
    # LC -> fitted curve (0,255,0)
    print(np.shape(ORI_IMGS))
    
elif VALID_TEST == 'test':
    file_num = "050"
    YOLO_IMG_PATH =  '/home/bono/Desktop/mellab_project/data/set1_inhaHos/detect_result/' + file_num +'/'+'detect'+'/'
    YOLO_TXT_PATH =  '/home/bono/Desktop/mellab_project/data/set1_inhaHos/detect_result/'+ file_num +'/'+'detect_txt'+'/'
    ORI_IMG_PATH =  '/home/bono/Desktop/mellab_project/data/set1_inhaHos/proc_data/' + file_num +'/'
    
    INFERENCE_SHAPE  = (64,64)
    
    # processing
    NUM_YOLO_RESULT = count_num_files(YOLO_IMG_PATH)
    SHAPE_YOLO_RESULT = get_shape_of_imgs(YOLO_IMG_PATH)
    print(SHAPE_YOLO_RESULT)
    YOLO_IMGS = load_yolo_imgs(YOLO_IMG_PATH, NUM_YOLO_RESULT, SHAPE_YOLO_RESULT) 
    ORI_IMGS = load_yolo_imgs(ORI_IMG_PATH, NUM_YOLO_RESULT, SHAPE_YOLO_RESULT)

    YOLO_RESULT_TABLE = load_and_classify_yolo_txt(YOLO_TXT_PATH)

    # Result image (BMO + LC, 3ch)
    RESULT_IMGS = ORI_IMGS.copy()
    RESULT_IMGS_MASKS = np.zeros((NUM_YOLO_RESULT, SHAPE_YOLO_RESULT[0], SHAPE_YOLO_RESULT[1], SHAPE_YOLO_RESULT[2]), dtype=np.uint16)

    # BMO -> Center points (255,0,0)
    # LC -> fitted curve (0,255,0)
    print(np.shape(ORI_IMGS))


(500, 760, 3)
(20, 500, 760, 3)


YOLO_RESULT_TABLE Inference fine segmentor
- Input : padded yolo results array
- Description : 
    - bmo -> center crop (64 x 64)
    - lc -> overlap-tile 이용해야할듯
    - overlap-tile inference
    - reconstruction images
- Output : Inferenced bmo, lc patch coordinate

#### BMO processing steps

In [21]:
# Load model
config = configparser.RawConfigParser()
config.read('configuration.txt')
name_experiment = config.get('experiment name', 'name')
path_experiment = './'+'result/'+ name_experiment
best_last = config.get('testing settings', 'best_last')
model = model_from_json(open(path_experiment+'/'+ name_experiment +'_architecture.json').read())
print(path_experiment)
model.load_weights(path_experiment+'/'+best_last+'_weights.h5')

def bmo_centerCrop_patch(ori_imgs,num_imgs, patch_shape ,yolo_table, save_folder, debug_flag):
    bmo01_patch = np.zeros((num_imgs, patch_shape[0], patch_shape[1], 1))
    bmo02_patch = np.zeros((num_imgs, patch_shape[0], patch_shape[1], 1))
    
    bmo01_init_points_info = []
    bmo02_init_points_info = []
    
    print('shape ori imgs : {} shape patch01 : {} shape patch02 : {}'.format(np.shape(ori_imgs), 
                                                                             np.shape(bmo01_patch),
                                                                            np.shape(bmo02_patch)))
    
    for idx in range(num_imgs):
        bmo01_xmin = yolo_table.iloc[idx]['bmo1_xmin'].astype(int)
        bmo01_ymin = yolo_table.iloc[idx]['bmo1_ymin'].astype(int)
        bmo01_xmax = yolo_table.iloc[idx]['bmo1_xmax'].astype(int)
        bmo01_ymax = yolo_table.iloc[idx]['bmo1_ymax'].astype(int)
        bmo01_center_x = int((bmo01_xmax + bmo01_xmin)/2) ;bmo01_center_y = int((bmo01_ymax + bmo01_ymin)/2)
        #print('bmo01 center x : {} bmo01 center y : {}'.format(bmo01_center_x, bmo01_center_y))
        
        bmo02_xmin = yolo_table.iloc[idx]['bmo2_xmin'].astype(int)
        bmo02_ymin = yolo_table.iloc[idx]['bmo2_ymin'].astype(int)
        bmo02_xmax = yolo_table.iloc[idx]['bmo2_xmax'].astype(int)
        bmo02_ymax = yolo_table.iloc[idx]['bmo2_ymax'].astype(int)
        bmo02_center_x = int((bmo02_xmax + bmo02_xmin)/2) ;bmo02_center_y = int((bmo02_ymax + bmo02_ymin)/2)
        #print('bmo01 center x : {} bmo01 center y : {}'.format(bmo02_center_x, bmo02_center_y))
        bmo01_init_points_info.append((bmo01_center_y-32,bmo01_center_x-32 ))
        bmo02_init_points_info.append((bmo02_center_y-32,bmo02_center_x-32 ))
        
        ori_bmo01_crop = ori_imgs[idx, bmo01_center_y-32 : bmo01_center_y+32, bmo01_center_x-32 : bmo01_center_x+32,0]
        ori_bmo02_crop = ori_imgs[idx, bmo02_center_y-32 : bmo02_center_y+32, bmo02_center_x-32 : bmo02_center_x+32,0]
        
        ori_bmo01_crop = np.expand_dims(ori_bmo01_crop,-1)
        ori_bmo02_crop = np.expand_dims(ori_bmo02_crop,-1)
        
        bmo01_patch[idx] = ori_bmo01_crop
        bmo02_patch[idx] = ori_bmo02_crop
        
        if debug_flag == True:
            copy_img = ori_imgs[idx].copy()
            
            tempRC01 = tuple([bmo01_center_x-32,bmo01_center_y-32])
            tempRC02 = tuple([bmo02_center_x-32,bmo02_center_y-32])
            
            cv2.circle(copy_img,tempRC01, 3,(0,0,255))
            cv2.circle(copy_img,tempRC02, 3,(0,0,255))
            
            cv2.imwrite(save_folder+'center_crop_debug_'+str(idx)+'_.png', copy_img)

        
    return bmo01_patch, bmo02_patch, bmo01_init_points_info, bmo02_init_points_info

def bmo_groundTruth_patch(gt_imgs,num_imgs, patch_shape ,yolo_table, save_folder, debug_flag):
    #ori_img = ground_Truth
    left_bmo_patch = np.zeros((num_imgs, patch_shape[0], patch_shape[1], 1))
    right_bmo_patch = np.zeros((num_imgs, patch_shape[0], patch_shape[1], 1))
    
    left_bmo_init_points_info = []
    right_bmo_init_points_info = []
    
    print('shape ori imgs : {} shape patch01 : {} shape patch02 : {}'.format(np.shape(gt_imgs), 
                                                                             np.shape(left_bmo_patch),
                                                                            np.shape(right_bmo_patch)))

    for idx in range(num_imgs):
        bmo01_xmin = yolo_table.iloc[idx]['bmo1_xmin'].astype(int)
        bmo01_ymin = yolo_table.iloc[idx]['bmo1_ymin'].astype(int)
        bmo01_xmax = yolo_table.iloc[idx]['bmo1_xmax'].astype(int)
        bmo01_ymax = yolo_table.iloc[idx]['bmo1_ymax'].astype(int)
        bmo01_center_x = int((bmo01_xmax + bmo01_xmin)/2) ;bmo01_center_y = int((bmo01_ymax + bmo01_ymin)/2)
        #print('bmo01 center x : {} bmo01 center y : {}'.format(bmo01_center_x, bmo01_center_y))
        
        bmo02_xmin = yolo_table.iloc[idx]['bmo2_xmin'].astype(int)
        bmo02_ymin = yolo_table.iloc[idx]['bmo2_ymin'].astype(int)
        bmo02_xmax = yolo_table.iloc[idx]['bmo2_xmax'].astype(int)
        bmo02_ymax = yolo_table.iloc[idx]['bmo2_ymax'].astype(int)
        bmo02_center_x = int((bmo02_xmax + bmo02_xmin)/2) ;bmo02_center_y = int((bmo02_ymax + bmo02_ymin)/2)
        #print('bmo01 center x : {} bmo01 center y : {}'.format(bmo02_center_x, bmo02_center_y))
        
        
        if bmo01_xmin > bmo02_xmin:
            print('reverse! : {} and {}'.format(bmo01_xmin, bmo02_xmin))
            left_bmo_init_points_info.append((bmo02_center_y-32,bmo02_center_x-32 ))
            right_bmo_init_points_info.append((bmo01_center_y-32,bmo01_center_x-32 ))
        else:
            left_bmo_init_points_info.append((bmo01_center_y-32,bmo01_center_x-32 ))
            right_bmo_init_points_info.append((bmo02_center_y-32,bmo02_center_x-32 ))
        
        ori_bmo01_crop = gt_imgs[idx, bmo01_center_y-32 : bmo01_center_y+32, bmo01_center_x-32 : bmo01_center_x+32,2]
        ori_bmo02_crop = gt_imgs[idx, bmo02_center_y-32 : bmo02_center_y+32, bmo02_center_x-32 : bmo02_center_x+32,2]
        ori_bmo01_crop = np.expand_dims(ori_bmo01_crop,-1)
        ori_bmo02_crop = np.expand_dims(ori_bmo02_crop,-1)
        
        left_bmo_patch[idx] = ori_bmo01_crop
        right_bmo_patch[idx] = ori_bmo02_crop
        
        if bmo01_xmin > bmo02_xmin:
            left_bmo_patch[idx] = ori_bmo02_crop
            right_bmo_patch[idx] = ori_bmo01_crop
        
        if debug_flag == True:
            #copy_img = gt_imgs[idx].copy()
            cv2.imwrite(save_folder+'gt_left_crop_debug_'+str(idx)+'_.png', left_bmo_patch[idx])
            cv2.imwrite(save_folder+'gt_right_crop_debug_'+str(idx)+'_.png', right_bmo_patch[idx])
        
    return left_bmo_patch, right_bmo_patch, left_bmo_init_points_info, right_bmo_init_points_info

def decoding(labels,class_num):
    '''
    input : patchs (ch first images)
            (ch, H, W)
    
    mapping example : 
        mapping = {
            (0,0,0) : 0,
            (255,0,0) : 1,
            (0,0,255) : 2
            }
    
    return patches (num, class-1,H, W)
    '''
    #print('[ch-wise one hot encoding] label shape : ', np.shape(labels))
    c, h, w = labels.shape
    decoded_labels = np.zeros((h,w,3)) # RGB format
    
    ch_cnt =0 
    temp_label = np.transpose(labels, (1,2,0))
    #print(np.shape(temp_label))

    '''
    mapping 순서 이슈
    '''
    #print('[ch wise one hot] shape label : ', np.shape(temp_label))
    #print('[ch wise one hot] shape encoded label : ',np.shape(encoded_labels))
    for k in range(class_num):
        '''
        temp label
            (H,W)
        '''
        row,col = np.nonzero(temp_label[:,:,k])
        #print(len(row))
        if k==0:
            class_01_info = (row, col)
        elif k==1:
            class_02_info = (row, col)
            
    decoded_labels[class_01_info] = (255,0,0)
    decoded_labels[class_02_info] = (0,255,0)
    
    return decoded_labels



./result/20_0621_fine_segmentor_180_Data_focal_gamma_07


In [22]:
name_experiment = 'final_paper_debug/'

In [23]:
# hyperparam
dir_checker('./pipe_result/'+str(name_experiment)+'/')
debug_flag = True
save_folder2 = './pipe_result/'+str(name_experiment)+'/center_crop/'
save_folder = './pipe_result/'+str(name_experiment)+'/'
dir_checker(save_folder)
dir_checker(save_folder2)

save_bmo_result = 'bmo_patch/'
dir_checker(save_folder + save_bmo_result)
N_visual = int(config.get('testing settings', 'num_group_visual'))

# bmo processing
bmo01_patch, bmo02_patch, BMO01_INIT_POINTS, BMO02_INIT_POINTS= bmo_centerCrop_patch(ORI_IMGS,NUM_YOLO_RESULT, INFERENCE_SHAPE, YOLO_RESULT_TABLE,
                                                                                    save_folder2 ,debug_flag)

if VALID_TEST == 'valid':
    left_bmoGT_patch, right_bmoGT_patch, L_BMO_GT_INIT_POINTS, R_BMO_GT_INIT_POINTS= bmo_groundTruth_patch(BMO_GROUND_TRUTH_IMGS,NUM_YOLO_RESULT, INFERENCE_SHAPE, YOLO_RESULT_TABLE,
                                                                                    save_folder2 ,debug_flag)

if debug_flag == True:
    for idx in range(np.shape(bmo01_patch)[0]):
        cv2.imwrite(save_folder + save_bmo_result +'bmo01_patch_'+str(idx)+'_.png', bmo01_patch[idx])
        cv2.imwrite(save_folder + save_bmo_result +'bmo02_patch_'+str(idx)+'_.png', bmo02_patch[idx])

bmo01_patch = np.transpose(bmo01_patch, (0,3,1,2)); bmo02_patch = np.transpose(bmo02_patch, (0,3,1,2))
bmo01_patch = bmo01_patch /255.0 ; bmo02_patch = bmo02_patch/255.0

predictions_bmo01 = model.predict(bmo01_patch, batch_size=64, verbose=2)
predictions_bmo02 = model.predict(bmo02_patch, batch_size=64, verbose=2)

print('pred bmo shape : ',np.shape(predictions_bmo01))

pred_patches_bmo01 = pred_to_imgs(predictions_bmo01, INFERENCE_SHAPE[0], INFERENCE_SHAPE[1], "original")
pred_patches_bmo02 = pred_to_imgs(predictions_bmo02, INFERENCE_SHAPE[0], INFERENCE_SHAPE[1], "original")

print('pred bmo shape : ',np.shape(pred_patches_bmo01))


pred_patches_thr_bmo01 = pred_to_imgs(predictions_bmo01, INFERENCE_SHAPE[0], INFERENCE_SHAPE[1], "threshold")
pred_patches_thr_bmo02 = pred_to_imgs(predictions_bmo02, INFERENCE_SHAPE[0], INFERENCE_SHAPE[1], "threshold")

pred_imgs_bmo01_class01 = pred_patches_bmo01[:,0]; pred_imgs_bmo01_class01 = np.expand_dims(pred_imgs_bmo01_class01, 1)
pred_imgs_bmo01_class02 = pred_patches_bmo01[:,1]; pred_imgs_bmo01_class02 = np.expand_dims(pred_imgs_bmo01_class02, 1)

pred_thr_bmo01_class01 = pred_patches_thr_bmo01[:,0]; pred_thr_bmo01_class01 = np.expand_dims(pred_thr_bmo01_class01, 1)
pred_thr_bmo01_class02 = pred_patches_thr_bmo01[:,1]; pred_thr_bmo01_class02 = np.expand_dims(pred_thr_bmo01_class02, 1)

pred_imgs_bmo02_class01 = pred_patches_bmo02[:,0]; pred_imgs_bmo02_class01 = np.expand_dims(pred_imgs_bmo02_class01, 1)
pred_imgs_bmo02_class02 = pred_patches_bmo02[:,1]; pred_imgs_bmo02_class02 = np.expand_dims(pred_imgs_bmo02_class02, 1)

pred_thr_bmo02_class01 = pred_patches_thr_bmo02[:,0]; pred_thr_bmo02_class01 = np.expand_dims(pred_thr_bmo02_class01, 1)
pred_thr_bmo02_class02 = pred_patches_thr_bmo02[:,1]; pred_thr_bmo02_class02 = np.expand_dims(pred_thr_bmo02_class02, 1)


if debug_flag == True:
    visualize(group_images(pred_imgs_bmo01_class01,N_visual),save_folder+"bmo01_class01")#.show()
    visualize(group_images(pred_imgs_bmo01_class02,N_visual),save_folder+"bmo01_class02")#.show()
    visualize(group_images(pred_thr_bmo01_class01,N_visual),save_folder+"thr_bmo01_class01")#.show()
    visualize(group_images(pred_thr_bmo01_class02,N_visual),save_folder+"thr_bmo01_class02")#.show()
    
    visualize(group_images(pred_imgs_bmo02_class01,N_visual),save_folder+"bmo02_class01")#.show()
    visualize(group_images(pred_imgs_bmo02_class02,N_visual),save_folder+"bmo02_class02")#.show()
    visualize(group_images(pred_thr_bmo02_class01,N_visual),save_folder+"thr_bmo02_class01")#.show()
    visualize(group_images(pred_thr_bmo02_class02,N_visual),save_folder+"thr_bmo02_class02")#.show()

    for i in range(pred_patches_bmo01.shape[0]):
        cv2.imwrite(save_folder+'pred_thr_bmo01_'+str(i)+'_.png', decoding(pred_patches_thr_bmo01[i],pred_patches_bmo01.shape[1]))
        cv2.imwrite(save_folder+'pred_thr_bmo02_'+str(i)+'_.png', decoding(pred_patches_thr_bmo02[i],pred_patches_bmo01.shape[1]))
 


already exist the folder in this path : ./pipe_result/final_paper_debug//
already exist the folder in this path : ./pipe_result/final_paper_debug//
already exist the folder in this path : ./pipe_result/final_paper_debug//center_crop/
already exist the folder in this path : ./pipe_result/final_paper_debug//bmo_patch/
shape ori imgs : (20, 500, 760, 3) shape patch01 : (20, 64, 64, 1) shape patch02 : (20, 64, 64, 1)
shape ori imgs : (20, 500, 760, 3) shape patch01 : (20, 64, 64, 1) shape patch02 : (20, 64, 64, 1)
reverse! : 396 and 237
reverse! : 523 and 402
reverse! : 610 and 488
reverse! : 513 and 316
reverse! : 431 and 293
reverse! : 371 and 205
reverse! : 499 and 344
reverse! : 469 and 295
reverse! : 351 and 217
reverse! : 449 and 311
reverse! : 532 and 383
reverse! : 344 and 211
pred bmo shape :  (20, 4096, 3)
pred imgs shape :  (20, 4096, 2)
pred imgs shape :  (20, 4096, 2)
pred bmo shape :  (20, 2, 64, 64)
pred imgs shape :  (20, 4096, 2)
pred imgs shape :  (20, 4096, 2)


#### BMO 
Bmo processing scheme

In [24]:
def bmo_max_connected_components(bmo_patches, num_img, shape_img, save_folder, debug_flag):
    bmo01_thr = (np.transpose(bmo_patches, (0,2,3,1)) * 255).astype(np.uint8)

    biggest_label = np.zeros((num_img, shape_img[0], shape_img[1]))
    biggest_lists = []
    
    for idx in range(num_img):
        row, col = np.nonzero(bmo01_thr[idx,:,:,0])
        num_bmo_label, labeled_bmo_map = cv2.connectedComponents(bmo01_thr[idx,:,:,0])

        temp_biggest_label = []

        if num_bmo_label != 1:
            for label_idx in range(1, num_bmo_label):
                temp_row, temp_col = np.where(labeled_bmo_map == label_idx)
                temp_biggest_label.append((len(temp_row), label_idx))
        else:
            temp_biggest_label.append((0,0))

        temp_biggest_label = sorted(temp_biggest_label, reverse=True)
        biggest_lists.append(temp_biggest_label[0])

    for idx in range(num_img):
        row, col = np.nonzero(bmo01_thr[idx,:,:,0])
        num_bmo_label, labeled_bmo_map = cv2.connectedComponents(bmo01_thr[idx,:,:,0])

        if biggest_lists[idx][1] != 0:
            biggest_row, biggest_col = np.where(labeled_bmo_map == biggest_lists[idx][1])
            biggest_label[idx, biggest_row, biggest_col] = 255
    
    if debug_flag ==True:
        for idx in range(num_img):
            cv2.imwrite(save_folder+'connected_comp'+str(idx)+'_.png', biggest_label[idx])
    
    return biggest_label

def bmo_segmentation_reports(pred_bmo01, pred_bmo02, left_gt, right_gt,save_folder):
    from sklearn.metrics import roc_curve
    from sklearn.metrics import roc_auc_score
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import precision_recall_curve
    from sklearn.metrics import jaccard_similarity_score
    from sklearn.metrics import f1_score
    
    left_mean_acc_lists = []
    right_mean_acc_lists = []
    
    for idx in range(np.shape(pred_bmo01)[0]):
        pred01_row, pred01_col = np.where(pred_bmo01[idx]  == 255.0)
        pred02_row, pred02_col =  np.where(pred_bmo02[idx]  == 255.0)

        if np.mean(pred01_col) > np.mean(pred02_col):
            left_pred = pred_bmo02[idx]
            right_pred = pred_bmo01[idx]
        else:
            left_pred = pred_bmo01[idx]
            right_pred = pred_bmo02[idx]
            
        left_pred = (left_pred>200)
        right_pred = (right_pred >200)
        
        left_each_gt = (left_gt[idx,:,:,0] > 200)
        right_each_gt = (right_gt[idx,:,:,0] > 200)
        print(np.shape(left_each_gt))
        print('left pred, gt max : {} {}'.format(np.max(left_pred), np.max(left_each_gt)))

        flat_left_pred = left_pred.flatten()
        flat_right_pred = right_pred.flatten()

        flat_left_gt = left_each_gt.flatten()
        flat_right_gt = right_each_gt.flatten()

        l_table = segmentation_reports(confusion_matrix(flat_left_pred, flat_left_gt), 'left', idx)
        r_table = segmentation_reports(confusion_matrix(flat_right_pred, flat_right_gt), 'right', idx)
        
        
        left_intersection = np.sum((flat_left_pred * flat_left_gt))
        left_dice = ((2* left_intersection)+1) / ((np.sum(flat_left_pred) + np.sum(flat_left_gt)) +1)
        right_intersection = np.sum((flat_right_pred * flat_right_gt))
        right_dice = ((2* right_intersection)+1) / ((np.sum(flat_right_pred) + np.sum(flat_right_gt)) +1)
        l_table["Dice"] = left_dice
        r_table["Dice"] = right_dice
        
        left_mean_acc_lists.append(l_table["Accuracy"])
        right_mean_acc_lists.append(r_table["Accuracy"])
        
        lr_table = pd.concat([l_table, r_table], ignore_index=True)
        
        if idx == 0:
            prev_lr_table = lr_table
        else:
            lr_table = pd.concat([lr_table, prev_lr_table], ignore_index=True)
            prev_lr_table = lr_table
        
        debug_left_img = np.zeros((64,64,3), dtype=np.uint8)
        debug_right_img = np.zeros((64,64,3), dtype=np.uint8)
        
        debug_l_pred_row, debug_l_pred_col = np.where(left_pred  == True)
        debug_r_pred_row, debug_r_pred_col = np.where(right_pred  == True)
        debug_l_gt_row, debug_l_gt_col = np.where(left_each_gt  == True)
        debug_r_gt_row, debug_r_gt_col = np.where(right_each_gt  == True)
        
        debug_left_img[debug_l_pred_row, debug_l_pred_col,:] = (255,0,0)
        debug_left_img[debug_l_gt_row, debug_l_gt_col,:] = (0,0,255)
        debug_right_img[debug_r_pred_row, debug_r_pred_col,:] = (255,0,0)
        debug_right_img[debug_r_gt_row, debug_r_gt_col,:] = (0,0,255)
                
        cv2.imwrite(save_folder+'left_diff_'+str(idx)+'.png',debug_left_img)
        cv2.imwrite(save_folder+'right_diff_'+str(idx)+'.png',debug_right_img)
        
    print('mean left acc : ',np.mean(left_mean_acc_lists))
    print('mean right acc : ', np.mean(right_mean_acc_lists))
    #print(lr_table)
    return lr_table
    
def segmentation_reports(confusion_matrix,left_right,index):
    seg_reports_table = pd.DataFrame(columns=['Class','Specificity','Sensitivity','Precision', 'Recall', 'Accuracy','F1 Score', 'IoU'])

    tp, fp = confusion_matrix[0]
    fn, tn = confusion_matrix[1]
    SP = tn / (tn+fp) #specificity
    SE = tp / (tp+fn)#sensitivity
    PR = tp / (tp+fp) #precision
    RE = tp / (tp + fn) #Recall
    ACC = (tp + tn) / (tp+fp+fn+tn) #accuracy
    F1 = 2 *( (RE * PR) / (RE + PR)) #f1 score
    IoU = tp / (tp + fp + fn)
    Dice = 2*tp / (2*tp+fp+fn)
    
    seg_reports_table.loc[0,'Class'] = left_right+'_BMO_'+str(index)
    seg_reports_table.loc[0,'Specificity'] = SP; seg_reports_table.loc[0,'Sensitivity'] = SE
    seg_reports_table.loc[0,'Precision'] = PR; seg_reports_table.loc[0,'Recall'] = RE
    seg_reports_table.loc[0,'Accuracy'] = ACC; seg_reports_table.loc[0,'F1 Score'] = F1
    seg_reports_table.loc[0,'IoU'] = IoU;
    
    return seg_reports_table
    
    

def get_bmo_center_points(processed_bmo, num_img, shape_img, save_folder, debug_flag):
    bmo_center_points = np.zeros((num_img, shape_img[0], shape_img[1]))
    center_points_lists = [] 
    
    for idx in range(num_img):
        bmo_row, bmo_col = np.nonzero(processed_bmo[idx])
        if len(bmo_row) !=0 :
            mean_row = int(np.mean(bmo_row)); mean_col = int(np.mean(bmo_col))
            center_points_lists.append((mean_row, mean_col))
            bmo_center_points[idx, mean_row, mean_col] = 255
        else:
            center_points_lists.append((0, 0))
            
    
    if debug_flag == True:
        copy_img = np.zeros((num_img,shape_img[0], shape_img[1], 3))
        for idx in range(num_img):
            bmo_row, bmo_col = np.nonzero(processed_bmo[idx])
            if len(bmo_row) !=0 :
                mean_row = int(np.mean(bmo_row)); mean_col = int(np.mean(bmo_col))
                tempRC = tuple([mean_col, mean_row])
                cv2.circle(copy_img[idx],tempRC, 5,(0,255,0))
            else:
                pass
            
        for idx in range(num_img):
            cv2.imwrite(save_folder+'center_points'+str(idx)+'_.png', copy_img[idx])
        
    return bmo_center_points, center_points_lists

def bmo_reconstruction_oct(ori_imgs,num_img ,center_bmo01, center_bmo02, bmo01_init_points, bmo02_init_points, save_folder, debug_flag):
    recon_imgs = ori_imgs.copy()
    recon_bmo01_center_points_lists = []
    recon_bmo02_center_points_lists = []
    
    for idx in range(num_img):
        recon_bmo01_center_row = bmo01_init_points[idx][0] +  center_bmo01[idx][0] 
        recon_bmo01_center_col = bmo01_init_points[idx][1] +  center_bmo01[idx][1] 
        recon_bmo01_center_points_lists.append((recon_bmo01_center_row, recon_bmo01_center_col)) 
        
        recon_bmo02_center_row = bmo02_init_points[idx][0] +  center_bmo02[idx][0] 
        recon_bmo02_center_col = bmo02_init_points[idx][1] +  center_bmo02[idx][1]  
        recon_bmo02_center_points_lists.append((recon_bmo02_center_row, recon_bmo02_center_col)) 
        
        if debug_flag == True:
            temp_bmo01_center = tuple([recon_bmo01_center_col, recon_bmo01_center_row])
            temp_bmo02_center = tuple([recon_bmo02_center_col, recon_bmo02_center_row])
        
            cv2.circle(recon_imgs[idx],temp_bmo01_center, 3,(0,0,255))
            cv2.circle(recon_imgs[idx],temp_bmo02_center, 3,(0,0,255))
            
            cv2.imwrite(save_folder+'recon_img_'+str(idx)+'_.png', recon_imgs[idx])
        
    return recon_bmo01_center_points_lists, recon_bmo02_center_points_lists
        

In [25]:
# hyperparam
save_folder = './pipe_result/'+str(name_experiment)+'/processed_bmo/'
save_folder2 = './pipe_result/'+str(name_experiment)+'/center_point/'
save_folder3 = './pipe_result/'+str(name_experiment)+'/bmo_recon/'
save_folder4 = './pipe_result/'+str(name_experiment)+'/debug_pred_gt/'


dir_checker(save_folder)
dir_checker(save_folder2)
dir_checker(save_folder3)
dir_checker(save_folder4)

# processing
processed_bmo01 = bmo_max_connected_components(pred_thr_bmo01_class01, NUM_YOLO_RESULT,
                                                INFERENCE_SHAPE, save_folder, debug_flag)
processed_bmo02 = bmo_max_connected_components(pred_thr_bmo02_class01, NUM_YOLO_RESULT,
                                                INFERENCE_SHAPE, save_folder, debug_flag)


if VALID_TEST=='valid':
    LR_SEG_REPORTS= bmo_segmentation_reports(processed_bmo01, processed_bmo02, left_bmoGT_patch, right_bmoGT_patch,save_folder4)
    print(LR_SEG_REPORTS)
center_bmo01_img, center_bmo01_points = get_bmo_center_points(processed_bmo01, NUM_YOLO_RESULT, INFERENCE_SHAPE, 
                                     save_folder2, debug_flag)
center_bmo02_img, center_bmo02_points = get_bmo_center_points(processed_bmo02, NUM_YOLO_RESULT, INFERENCE_SHAPE, 
                                     save_folder2, debug_flag)

recon_bmo01_center,recon_bmo02_center= bmo_reconstruction_oct(ORI_IMGS,NUM_YOLO_RESULT,
                                                               center_bmo01_points, center_bmo02_points, 
                                                               BMO01_INIT_POINTS, BMO02_INIT_POINTS, 
                                                               save_folder3, debug_flag)


already exist the folder in this path : ./pipe_result/final_paper_debug//processed_bmo/
already exist the folder in this path : ./pipe_result/final_paper_debug//center_point/
already exist the folder in this path : ./pipe_result/final_paper_debug//bmo_recon/
already exist the folder in this path : ./pipe_result/final_paper_debug//debug_pred_gt/
(64, 64)
left pred, gt max : True True
(64, 64)
left pred, gt max : True True
(64, 64)
left pred, gt max : True True
(64, 64)
left pred, gt max : True True
(64, 64)
left pred, gt max : True True
(64, 64)
left pred, gt max : True True
(64, 64)
left pred, gt max : True True
(64, 64)
left pred, gt max : True True
(64, 64)
left pred, gt max : True True
(64, 64)
left pred, gt max : True True
(64, 64)
left pred, gt max : True True
(64, 64)
left pred, gt max : True True
(64, 64)
left pred, gt max : True True
(64, 64)
left pred, gt max : True True
(64, 64)
left pred, gt max : True True
(64, 64)
left pred, gt max : True True
(64, 64)
left pred, gt max : 

In [26]:
print(recon_bmo01_center[0])
print(recon_bmo01_center[0][0], recon_bmo01_center[0][1])
temp_coord= np.where(np.all(RESULT_IMGS[0] == (0,0,255), axis=-1))
print(temp_coord)

temp = RESULT_IMGS.copy()
cv2.line(temp[0], (recon_bmo01_center[0][1], recon_bmo01_center[0][0]), (recon_bmo02_center[0][1], recon_bmo02_center[0][0]), (255,255,0), 2 )


(179, 330)
179 330
(array([], dtype=int64), array([], dtype=int64))


array([[[ 0,  0,  0],
        [26, 26, 26],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [44, 44, 44],
        [ 4,  4,  4],
        ...,
        [ 3,  3,  3],
        [ 4,  4,  4],
        [ 8,  8,  8]],

       [[ 0,  0,  0],
        [47, 47, 47],
        [14, 14, 14],
        ...,
        [ 1,  1,  1],
        [ 6,  6,  6],
        [10, 10, 10]],

       ...,

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]]], dtype=uint8)

In [27]:
def load_groundTruth_extract_bmo_centerPoints(groundTruth_img_path,num_imgs,shape_imgs):
    img_h, img_w = shape_imgs[0], shape_imgs[1]
    groundTruth_imgs = np.zeros((num_imgs, img_h, img_w,3),dtype=np.uint8)    
    idx =0 
    left_center_points_lists = [] 
    right_center_points_lists = [] 
    print('gt shape : ',np.shape(groundTruth_imgs))
      
    for count, filename in enumerate(sorted(os.listdir(groundTruth_img_path)), start=0):
        if filename.startswith(".ipynb") == False:
            temp_img = np.zeros((img_h, img_w,1),dtype=np.uint8)    
            
            temp = cv2.imread(groundTruth_img_path + filename)
            groundTruth_imgs[idx] = temp.astype(np.uint8)
            
            row,col = np.where(np.all(temp == (0,0,255), axis = -1))
            temp_img[row,col] = 255
            
            num_bmo_label, labeled_bmo_map = cv2.connectedComponents(temp_img)
            bmo01_row, bmo01_col = np.where(labeled_bmo_map == 1)
            bmo02_row, bmo02_col = np.where(labeled_bmo_map == 2)
            
            mean_bmo01_col = np.mean(bmo01_col)
            mean_bmo02_col = np.mean(bmo02_col)
            
            if mean_bmo01_col > mean_bmo02_col:
                # bmo01 -> right / bmo02 -> left
                
                left_mean_row = int(np.mean(bmo02_row)); left_mean_col = int(np.mean(bmo02_col))
                right_mean_row = int(np.mean(bmo01_row)); right_mean_col = int(np.mean(bmo01_col))
                
                left_center_points_lists.append((left_mean_row, left_mean_col))
                right_center_points_lists.append((right_mean_row, right_mean_col))
                
            else:
                
                left_mean_row = int(np.mean(bmo01_row)); left_mean_col = int(np.mean(bmo01_col))
                right_mean_row = int(np.mean(bmo02_row)); right_mean_col = int(np.mean(bmo02_col))
                
                left_center_points_lists.append((left_mean_row, left_mean_col))
                right_center_points_lists.append((right_mean_row, right_mean_col))
            
            idx = idx+1
            
    return groundTruth_imgs, left_center_points_lists, right_center_points_lists

def calc_dist_gt_pred_center(left_ground_center, right_ground_center, bmo01_pred_center, bmo02_pred_center,
                             num_img,ori_img ,save_folder, debug_flag):
    
    _x_convert = 12.5000
    _y_convert = 3.9216
    
    left_l1_diff = np.zeros(num_img, dtype=np.float32)
    right_l1_diff = np.zeros(num_img, dtype=np.float32)
    
    left_l2_diff = np.zeros(num_img, dtype=np.float32) # euclidian
    right_l2_diff = np.zeros(num_img, dtype=np.float32)
    
    grd_l2_dist = np.zeros(num_img, dtype=np.float32)
    pred_l2_dist = np.zeros(num_img, dtype=np.float32)
    
    ym_left_l1_diff = np.zeros(num_img, dtype=np.float32)
    ym_right_l1_diff = np.zeros(num_img, dtype=np.float32)
    
    ym_left_l2_diff = np.zeros(num_img, dtype=np.float32) # euclidian
    ym_right_l2_diff = np.zeros(num_img, dtype=np.float32)
    
    ym_grd_l2_dist = np.zeros(num_img, dtype=np.float32)
    ym_pred_l2_dist = np.zeros(num_img, dtype=np.float32)
    
    for idx in range(num_img):
        if bmo01_pred_center[idx][1] > bmo02_pred_center[idx][1]:
            left_pred_row = bmo02_pred_center[idx][0]; left_pred_col = bmo02_pred_center[idx][1]
            right_pred_row = bmo01_pred_center[idx][0]; right_pred_col = bmo01_pred_center[idx][1]
        else:
            left_pred_row = bmo01_pred_center[idx][0]; left_pred_col = bmo01_pred_center[idx][1]
            right_pred_row = bmo02_pred_center[idx][0]; right_pred_col = bmo02_pred_center[idx][1]
            
        left_ground_row = left_ground_center[idx][0]; left_ground_col = left_ground_center[idx][1] 
        right_ground_row = right_ground_center[idx][0]; right_ground_col = right_ground_center[idx][1]
        
        # 0 = left, 1 = right
        left_gt = np.array((left_ground_row, left_ground_col))
        right_gt = np.array((right_ground_row, right_ground_col))
        left_pred = np.array((left_pred_row, left_pred_col))
        right_pred = np.array((right_pred_row, right_pred_col))
        
        left_l1_diff[idx] = abs(left_ground_row -left_pred_row) + abs(left_ground_col - left_pred_col)
        right_l1_diff[idx] = abs(right_ground_row - right_pred_row) + abs(right_ground_col - right_pred_col)
        
        left_l2_diff[idx] = np.linalg.norm(left_gt - left_pred)
        right_l2_diff[idx] = np.linalg.norm(right_gt - right_pred)
        grd_l2_dist[idx] = np.linalg.norm(right_gt- left_gt)
        pred_l2_dist[idx] = np.linalg.norm(right_pred - left_pred)
        
        
        # ym distance
        ym_left_l1_diff[idx] = abs(left_ground_row -left_pred_row) * _y_convert + abs(left_ground_col - left_pred_col) * _x_convert
        ym_right_l1_diff[idx] = abs(right_ground_row - right_pred_row) * _y_convert + abs(right_ground_col - right_pred_col) * _x_convert
        
        ym_left_l2_diff[idx] = np.sqrt(np.square((left_gt[0] - left_pred[0]) * _y_convert) + np.square((left_gt[1] - left_pred[1]) * _x_convert) )
        ym_right_l2_diff[idx] = np.sqrt(np.square((right_gt[0] - right_pred[0]) * _y_convert) + np.square((right_gt[1] - right_pred[1]) * _x_convert) )
        
        ym_grd_l2_dist[idx] = np.sqrt(np.square((left_gt[0] - right_gt[0]) * _y_convert) + np.square((left_gt[1] - right_gt[1]) * _x_convert) )
        ym_pred_l2_dist[idx] = np.sqrt(np.square((left_pred[0] - right_pred[0]) * _y_convert) + np.square((left_pred[1] - right_pred[1]) * _x_convert) )
        
        #left_l2_diff[idx] = ((((left_ground_row -left_pred_row) **2 ) * _y_convert) + (((left_ground_col - left_pred_col) **2)*_x_convert))**1/2
        #right_l2_diff[idx] = ((((right_ground_row -right_pred_row) **2 ) *_y_convert) + (((right_ground_col - right_pred_col) **2 )*_x_convert))**1/2
        
        if debug_flag == True:
            debug_ori_img = ori_img[idx].copy()
            debug_left_gt_center = tuple([left_ground_col, left_ground_row])
            debug_right_gt_center = tuple([right_ground_col, right_ground_row])
            debug_left_pred_center = tuple([left_pred_col, left_pred_row])
            debug_right_pred_center = tuple([right_pred_col, right_pred_row])
            
        
            cv2.circle(debug_ori_img,debug_left_gt_center, 3,(255,0,0))
            cv2.circle(debug_ori_img,debug_right_gt_center, 3,(255,0,0))
            print('save circle')
            cv2.imwrite(save_folder+'ground_truth_'+str(idx)+'_.png', debug_ori_img)
            
            cv2.circle(debug_ori_img,debug_left_pred_center, 3,(0,0,255))
            cv2.circle(debug_ori_img,debug_right_pred_center, 3,(0,0,255))
            
            cv2.imwrite(save_folder+'diff_center_'+str(idx)+'_.png', debug_ori_img)
            
    
    print('left l1 mean dist : {} right l1 mean dist : {}'.format(np.mean(left_l1_diff), np.mean(right_l1_diff)))
    print('left l1 var dist : {} right l1 var dist : {}\n\n'.format(np.std(left_l1_diff), np.std(right_l1_diff)))
    
    
    print('left l2 mean dist : {} right l2 mean dist : {}'.format(np.mean(left_l2_diff), np.mean(right_l2_diff)))
    print('left l2 var dist : {} right l2 var dist : {}\n\n'.format(np.std(left_l2_diff), np.std(right_l2_diff)))
    
    print('[ym] left l1 mean dist : {} right l1 mean dist : {}'.format(np.mean(ym_left_l1_diff), np.mean(ym_right_l1_diff)))
    print('[ym] left l1 var dist : {} right l1 var dist : {}\n\n'.format(np.std(ym_left_l1_diff), np.std(ym_right_l1_diff)))
    
    
    print('[ym] left l2 mean dist : {} right l2 mean dist : {}'.format(np.mean(ym_left_l2_diff), np.mean(ym_right_l2_diff)))
    print('[ym] left l2 var dist : {} right l2 var dist : {}\n\n'.format(np.std(ym_left_l2_diff), np.std(ym_right_l2_diff)))
    
    l1_table = pd.DataFrame(columns=['CLASS','Mean_l1_distance','Std_l1_distance', 'Mean_ym_l1_distance', 'Std_ym_l1_distance'])
    l1_table.loc[0,'CLASS' ] = 'LEFT_BMO'
    l1_table.loc[0,'Mean_l1_distance' ] = np.mean(left_l1_diff); l1_table.loc[0,'Std_l1_distance' ] = np.std(left_l1_diff)
    l1_table.loc[0,'Mean_ym_l1_distance' ] = np.mean(ym_left_l1_diff); l1_table.loc[0,'Std_ym_l1_distance' ] = np.std(ym_left_l1_diff)
    
    l1_table.loc[1,'CLASS' ] = 'RIGHT_BMO' 
    l1_table.loc[1,'Mean_l1_distance' ] = np.mean(right_l1_diff); l1_table.loc[1,'Std_l1_distance' ] = np.std(right_l1_diff)
    l1_table.loc[1,'Mean_ym_l1_distance' ] = np.mean(ym_right_l1_diff); l1_table.loc[1,'Std_ym_l1_distance' ] = np.std(ym_right_l1_diff)
    
    
    l2_table = pd.DataFrame(columns=['CLASS','Mean_l2_distance','Std_l2_distance', 'Mean_ym_l2_distance', 'Std_ym_l2_distance'])
    l2_table.loc[0,'CLASS' ] = 'LEFT_BMO'
    l2_table.loc[0,'Mean_l2_distance' ] = np.mean(left_l2_diff); l2_table.loc[0,'Std_l2_distance' ] = np.std(left_l2_diff)
    l2_table.loc[0,'Mean_ym_l2_distance' ] = np.mean(ym_left_l2_diff); l2_table.loc[0,'Std_ym_l2_distance' ] = np.std(ym_left_l2_diff)
    
    l2_table.loc[1,'CLASS' ] = 'RIGHT_BMO' 
    l2_table.loc[1,'Mean_l2_distance' ] = np.mean(right_l2_diff); l2_table.loc[1,'Std_l2_distance' ] = np.std(right_l2_diff)
    l2_table.loc[1,'Mean_ym_l2_distance' ] = np.mean(ym_right_l2_diff); l2_table.loc[1,'Std_ym_l2_distance' ] = np.std(ym_right_l2_diff)
    
    bmo_l2_diff = np.concatenate((left_l2_diff, right_l2_diff), axis = 0)
    bmo_ym_l2_diff = np.concatenate((ym_left_l2_diff, ym_right_l2_diff), axis = 0)

    
    l2_table.loc[2,'CLASS' ] = 'BMO'
    l2_table.loc[2,'Mean_l2_distance' ] = np.mean(bmo_l2_diff)
    l2_table.loc[2,'Std_l2_distance' ] = np.std(bmo_l2_diff)
    l2_table.loc[2,'Mean_ym_l2_distance' ] = np.mean(bmo_ym_l2_diff)
    l2_table.loc[2,'Std_ym_l2_distance' ] = np.std(bmo_ym_l2_diff)
    
    
    
    bmo_l2_table = pd.DataFrame(columns=['CLASS','Mean_bmo_l2_distance', 'Mean_ym_bmo_l2_distance'])
    bmo_l2_table.loc[0,'CLASS' ] = 'GROUND_TRUTH'
    bmo_l2_table.loc[0,'Mean_bmo_l2_distance' ] = np.mean(grd_l2_dist); bmo_l2_table.loc[0,'Mean_ym_bmo_l2_distance' ] = np.mean(ym_grd_l2_dist)
    
    bmo_l2_table.loc[1,'CLASS' ] = 'PREDICTION'
    bmo_l2_table.loc[1,'Mean_bmo_l2_distance' ] = np.mean(pred_l2_dist); bmo_l2_table.loc[0,'Mean_ym_bmo_l2_distance' ] = np.mean(ym_pred_l2_dist)
    
    return l1_table, l2_table, bmo_l2_table

In [28]:
# hyperparam
save_folder = './pipe_result/'+str(name_experiment)+'/bmo_final_diff/'
dir_checker(save_folder)

# processing
GROUND_TRUTH_IMG, LEFT_GROUND_CENTER,RIGHT_GROUND_CENTER = load_groundTruth_extract_bmo_centerPoints(GROUND_TRUTH_PATH, NUM_YOLO_RESULT, SHAPE_YOLO_RESULT)

l1_table, l2_table, bmo_l2_table = calc_dist_gt_pred_center(LEFT_GROUND_CENTER,RIGHT_GROUND_CENTER, recon_bmo01_center,recon_bmo02_center,NUM_YOLO_RESULT,ORI_IMGS, save_folder, debug_flag)

already exist the folder in this path : ./pipe_result/final_paper_debug//bmo_final_diff/
gt shape :  (20, 500, 760, 3)
save circle
save circle
save circle
save circle
save circle
save circle
save circle
save circle
save circle
save circle
save circle
save circle
save circle
save circle
save circle
save circle
save circle
save circle
save circle
save circle
left l1 mean dist : 6.699999809265137 right l1 mean dist : 3.3499999046325684
left l1 var dist : 7.149126052856445 right l1 var dist : 4.952524662017822


left l2 mean dist : 5.045083045959473 right l2 mean dist : 2.6194605827331543
left l2 var dist : 5.417300701141357 right l2 var dist : 3.659019947052002


[ym] left l1 mean dist : 49.865325927734375 right l1 mean dist : 28.578481674194336
[ym] left l1 var dist : 47.05536651611328 right l1 var dist : 34.001163482666016


[ym] left l2 mean dist : 38.786231994628906 right l2 mean dist : 24.13015365600586
[ym] left l2 var dist : 34.303958892822266 right l2 var dist : 25.11441993713379


In [29]:
import six

def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')

    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)

    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in six.iteritems(mpl_table._cells):
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return fig,ax

In [ ]:
save_folder = './pipe_result/'+str(name_experiment)+'/'


l1_table
fig,ax = render_mpl_table(l1_table, header_columns=0, col_width=5.0)
fig.savefig(save_folder + 'bmo_l1_table.png')

In [ ]:
l2_table

fig,ax = render_mpl_table(l2_table, header_columns=0, col_width=5.0)
fig.savefig(save_folder + 'bmo_l2_table.png')

In [ ]:
bmo_l2_table

fig,ax = render_mpl_table(bmo_l2_table, header_columns=0, col_width=5.0)
fig.savefig(save_folder + 'each_left_right_bmo_l2_table.png')

In [ ]:
LR_SEG_REPORTS
fig,ax = render_mpl_table(LR_SEG_REPORTS, header_columns=0, col_width=5.0)
fig.savefig(save_folder + 'bmo_seg_reports.png')

In [ ]:
mean_seg_table = pd.DataFrame(columns=['Class','Accuracy(%)','F1 Score(%)', 'Dice(%)'])
mean_seg_table.loc[0,"Class"] = "BMO"
mean_seg_table.loc[0,"Accuracy(%)"] = np.mean(LR_SEG_REPORTS["Accuracy"])
mean_seg_table.loc[0,"F1 Score(%)"] = np.mean(LR_SEG_REPORTS["F1 Score"])
mean_seg_table.loc[0,"Dice(%)"] = np.mean(LR_SEG_REPORTS["Dice"])

fig,ax = render_mpl_table(mean_seg_table, header_columns=0, col_width=5.0)
fig.savefig(save_folder + 'final_mean_bmo_table.png')

#### LC processing  
이미지 한장한장 적용을 해줘야 할 것 같다. (혹은 list나 numpy.append 이용....)
1. overlap-tile prediction
overlap-tile 전략을 사용함에 있어서.   
    - **Number of images (each row, col)**   
        (image size - patch size) // (stride +1) 이는 stride 갯수만큼 이미지를 나누기 떄문  
    - **Total number of images**  
        (image height - patch height) // (stride +1) * (image width - patch width) // (stride +1)  
    - **Overlap-tile condition**  
        
        
2. reconstruct patch
3. get centerline
4. reconstruction to oct

In [ ]:
def padding_border_overlap(data, patch_h, patch_w,stride_h, stride_w):
    
    img_h = data.shape[0]
    img_w = data.shape[1]
    
    leftover_h = (img_h - patch_h) % stride_h
    leftover_w = (img_w - patch_w) % stride_w
    
    if(leftover_h != 0):
        print("\nthe side H is not compatible with the selected stride of " +str(stride_h))
        print("img_h " +str(img_h) + ", patch_h " +str(patch_h) + ", stride_h " +str(stride_h))
        print("(img_h - patch_h) MOD stride_h: " +str(leftover_h))
        print("So the H dim will be padded with additional " +str(stride_h - leftover_h) + " pixels")
        
        temp_full_imgs = np.zeros((img_h+(stride_h-leftover_h),img_w))
        temp_full_imgs[0:img_h,0:img_w] = data
        data = temp_full_imgs
        
    if(leftover_w != 0):
        print("\nthe side H is not compatible with the selected stride of " +str(stride_w))
        print("img_h " +str(img_w) + ", patch_h " +str(patch_w) + ", stride_h " +str(stride_w))
        print("(img_h - patch_h) MOD stride_h: " +str(leftover_w))
        print("So the H dim will be padded with additional " +str(stride_w - leftover_w) + " pixels")
        
        temp_full_imgs = np.zeros((data.shape[0],img_w+(stride_w - leftover_w)))
        temp_full_imgs[0:data.shape[0],0:img_w] = data
        data = temp_full_imgs
    print ("after paint border lc img shape : " +str(data.shape))
    new_h = np.shape(data)[0]; new_w = np.shape(data)[1]
    return data, new_h, new_w

def lc_overlap_tile_prediction(ori_imgs, deep_model,bbox_coord,stride,num_img, shape_inference, save_folder, debug_flag):
    lc_img = ori_imgs[bbox_coord[1] : bbox_coord[3], bbox_coord[0]:bbox_coord[2],1]
    img_h = np.shape(lc_img)[0]; img_w = np.shape(lc_img)[1]
    patch_h = shape_inference[0]; patch_w = shape_inference[1]
    stride_h = stride; stride_w = stride
    
    if debug_flag ==True:
        cv2.imwrite(save_folder+'lc_img.png', lc_img)
    
    
    print('before paint border lc img shape : {}'.format(np.shape(lc_img)))
    lc_img,new_h, new_w = padding_border_overlap(lc_img, patch_h, patch_w, stride_h, stride_w)
    print('new h : {} new w : {}'.format(new_h, new_w))
    
    img_h = new_h; img_w = new_w
    num_patch_img = ((img_h - patch_h)//stride_h +1) * ((img_w - patch_w)//stride_w +1)
    lc_patches = np.empty((num_patch_img, patch_h, patch_w))
    cnt =0 
    
    for h in range((img_h - patch_h)//stride_h +1):
        for w in range((img_w - patch_w)//stride_w +1): 
            lc_patches[cnt] = lc_img[h*stride_h : (h*stride_h)+patch_h, w*stride_w : (w*stride_w)+patch_w ]
            cnt = cnt+1
    
    print('lc patch shape after extract overlap : {}'.format(np.shape(lc_patches)))
    
    if debug_flag ==True:
        for idx in range(num_patch_img):
            #print(np.shape(lc_patches[idx]))
            cv2.imwrite(save_folder+'lc_patched'+str(idx)+'_.png', lc_patches[idx])
          
    lc_patches = np.expand_dims(lc_patches,-1)
    lc_patches = np.transpose(lc_patches, (0,3,1,2))
    lc_patches = lc_patches / 255.0
    lc_pred = deep_model.predict(lc_patches, batch_size=64, verbose=2)
    
    print('shape lc patches : {}'.format(np.shape(lc_pred)))
    
    lc_pred_thr = pred_to_imgs(lc_pred, shape_inference[0], shape_inference[1], "threshold")
    lc_pred = pred_to_imgs(lc_pred, shape_inference[0], shape_inference[1], "original")
    
    
    pred_lc_class01 = lc_pred[:,0]; pred_lc_class01 = np.expand_dims(pred_lc_class01, 1)
    pred_lc_class02 = lc_pred[:,1]; pred_lc_class02 = np.expand_dims(pred_lc_class02, 1)
    
    pred_lc_thr_class01 = lc_pred_thr[:,0]; pred_lc_thr_class01 = np.expand_dims(pred_lc_thr_class01, 1)
    pred_lc_thr_class02 = lc_pred_thr[:,1]; pred_lc_thr_class02 = np.expand_dims(pred_lc_thr_class02, 1)

    print('shape lc patches class : {}'.format(np.shape(pred_lc_class01)))
    
    if debug_flag == True:
        temp_pred_lc_class01 = pred_lc_class01.copy()
        temp_pred_lc_class02 = pred_lc_class02.copy()
        
        temp_pred_lc_class01 = np.transpose(temp_pred_lc_class01, (0,2,3,1))
        temp_pred_lc_class02 = np.transpose(temp_pred_lc_class02, (0,2,3,1))
        for idx in range(np.shape(pred_lc_class01)[0]):
            cv2.imwrite(save_folder + 'pred_lc_patches_class01_' + str(idx)+ '_.png',temp_pred_lc_class01[idx] * 255)
            cv2.imwrite(save_folder + 'pred_lc_patches_class02_' + str(idx)+ '_.png',temp_pred_lc_class02[idx]* 255)
        
    return pred_lc_class01, pred_lc_class02,pred_lc_thr_class01,pred_lc_thr_class02, new_h, new_w
        
    

def lc_reconstruction_avg(patch_lc_pred, shape_inference, new_h, new_w,stride_gap,thr_mode, save_folder , debug_flag ):
    lc_pred = patch_lc_pred
    img_h = new_h; img_w = new_w
    patch_h = shape_inference[0]; patch_w = shape_inference[1]
    stride_h = stride_gap; stride_w = stride_gap
    
    full_prob = np.zeros((lc_pred.shape[1],img_h,img_w))  #itialize to zero mega array with sum of Probabilities
    full_sum = np.zeros((lc_pred.shape[1],img_h,img_w))

    k =0 
    for h in range((img_h-patch_h)//stride_h+1):
        for w in range((img_w-patch_w)//stride_w+1):
            full_prob[:,h*stride_h:(h*stride_h)+patch_h,w*stride_w:(w*stride_w)+patch_w]+=lc_pred[k]
            full_sum[:,h*stride_h:(h*stride_h)+patch_h,w*stride_w:(w*stride_w)+patch_w]+=1
            k+=1
    
    print(np.min(full_sum))
    assert(k==lc_pred.shape[0])
    assert(np.min(full_sum)>=1.0)
    
    
    final_avg = full_prob/full_sum
    print (final_avg.shape)
    print(np.max(final_avg))
    assert(np.max(final_avg)<=1.0) #max value for a pixel is 1.0
    assert(np.min(final_avg)>=0.0) #min value for a pixel is 0.0
    
    if debug_flag ==True:
        temp_final_avg = np.transpose(final_avg,(1,2,0))
        if thr_mode == 'threshold':
            cv2.imwrite(save_folder+'reconst_thr_lc.png', temp_final_avg*255)
        elif thr_mode == 'original':
            cv2.imwrite(save_folder+'reconst_lc.png', temp_final_avg*255)

    return final_avg

def pred_reconstruction_oct(ori_img, lc_img, bbox, diff_h, diff_w, save_folder, debug_flag):
    lc_img = lc_img > 0.40
    row, col = np.nonzero(lc_img[0,:,:])
    
    add_row = bbox[1]; add_col = bbox[0]
    row = row + add_row
    col = col + add_col 
    
    result_recon_ori = ori_img.copy()
    result_recon_ori[row, col] = (0,255,0)
    
    cv2.imwrite(save_folder+'reconstruct_img.png',result_recon_ori)
    return result_recon_ori
    
def fillHole(im_in):
    im_floodfill = im_in.copy()

    # Mask used to flood filling.
    # Notice the size needs to be 2 pixels than the image.
    h, w = im_in.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)

    # Floodfill from point (0, 0)
    cv2.floodFill(im_floodfill, mask, (0,0), 255);

    # Invert floodfilled image
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)

    # Combine the two images to get the foreground.
    im_out = im_in | im_floodfill_inv

    return im_out


def post_proc_recon_img(recon_img,shape_img, connect_thr_val,img_idx,save_folder, debug_flag):
    temp_recon_img = np.zeros((shape_img[0], shape_img[1],1),dtype=np.uint8)
    pred_row, pred_col = np.where(np.all(recon_img == (0,255,0), axis= -1))
    temp_recon_img[pred_row, pred_col,0] = 255
    
    post_proc_label = np.zeros((np.shape(temp_recon_img)))

    #show_on_jupyter(temp_recon_img[:,:,0],'gray')
    temp = temp_recon_img[:,:,0]
    #temp = fillHole(temp_recon_img[:,:,0])
    if debug_flag == True:
        show_on_jupyter(temp,'gray')
    
    num_lc_label, labeled_lc_map = cv2.connectedComponents(temp)
    
    biggest_lists = []
    temp_biggest_label = []

    if num_lc_label != 1:
        for label_idx in range(1, num_lc_label):
            temp_row, temp_col = np.where(labeled_lc_map == label_idx)
            temp_biggest_label.append((len(temp_row), label_idx))
    else:
        temp_biggest_label.append((0,0))

    temp_biggest_label = sorted(temp_biggest_label, reverse=True)
    #print('labeled label idx : ',temp_biggest_label)
    
    num_lc_label, labeled_lc_map = cv2.connectedComponents(temp)
    
    for idx in range(num_lc_label-1):
        #print('idx of temp biggest : ',temp_biggest_label[idx])
        #print('temp biggest [idx][0] : {} [idx][1] : {}'.format(temp_biggest_label[idx][0], temp_biggest_label[idx][1]))
        if temp_biggest_label[idx][0] > connect_thr_val:
            post_row, post_col = np.where(labeled_lc_map == temp_biggest_label[idx][1])
            post_proc_label[post_row, post_col,0] = 255
        else:
            pass
        
    if debug_flag == True:
        cv2.imwrite(save_folder+'post_proc_lc'+ str(img_idx)+'.png', post_proc_label)
        show_on_jupyter(post_proc_label[:,:,0].astype(np.uint8), 'gray')

    return post_proc_label
    

    
def diff_spline_fitting_reconNori_img(ori_label,recon_img,shape_img,spline_order, ori_img, debug_flag):
    '''
    [TO-DO]
        - visualization spline fitting result (ori <-> recon)
        - Blend-altman plot
    '''
    
    from scipy.interpolate import interp1d
    import matplotlib.pyplot as plt
    from scipy.optimize import curve_fit
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.linear_model import LinearRegression
    from sklearn.pipeline import make_pipeline
    from sklearn.linear_model import Ridge


    
    
    
    _x_convert = 12.5000
    _y_convert = 3.9216
    
    temp_img = np.zeros((shape_img[0], shape_img[1], 3))
    label_row, label_col = np.where(np.all(ori_label == (255,0,0),axis=-1))
    
    #temp_recon = np.zeros((shape_img[0], shape_img[1], 3), dtype=np.float32)
    
    pred_row, pred_col = np.nonzero(recon_img[:,:,0])
    
    temp_img[label_row, label_col ,: ] = (255,0,0)
    temp_img[pred_row, pred_col , : ] = (0,255,0)
    temp_img = temp_img.astype(np.uint8)
    
    if debug_flag == True:
        show_on_jupyter(temp_img)
    temp_ori = ori_img.copy()
    temp_ori[label_row, label_col,:]= (255,0,0)
    temp_ori[pred_row, pred_col , : ] = (0,255,0)
    if debug_flag == True:
        show_on_jupyter(temp_ori)

    #print('label col shape : {} pred col shape: {}: '.format(label_col.shape, pred_col.shape))
    flat_label_col = label_col.reshape(-1,1)
    flat_label_row = label_row.reshape(-1,1)
    
    flat_pred_col = pred_col.reshape(-1,1)
    flat_pred_row = pred_row.reshape(-1,1)
    
    new_label_x = np.linspace(np.min(label_col), np.max(label_col),int(np.max(label_col) -np.min(label_col)+1))
    new_pred_x = np.linspace(np.min(pred_col), np.max(pred_col), int(np.max(pred_col) - np.min(pred_col)+1))
    
    #print('flat label col shape : {}'.format(flat_label_col.shape))

    '''
    numpy polyfit version
    
    label_regression_coef = np.polyfit(label_col, label_row ,spline_order)
    pred_regression_coef = np.polyfit(pred_col , pred_row,spline_order)
    print('polynomal coef : ', label_regression_coef)
    
    label_poly = np.poly1d(label_regression_coef)
    pred_poly = np.poly1d(pred_regression_coef)
    
    
    new_label_y = label_poly(new_label_x)
    new_pred_y = pred_poly(new_pred_x)
    
    print('nmpy label pred : {} \n\n shape : {} '.format(new_label_y,new_label_y.shape))
    '''
    
    #### sklearn poly fit experiment, (For apply the regularization term)
    label_polyreg = make_pipeline(
        PolynomialFeatures(degree=spline_order, include_bias=False),
        Ridge(alpha=0.01,solver='cholesky',normalize=True)
        )
    label_polyreg.fit(flat_label_col, flat_label_row)
    
    pred_polyreg = make_pipeline(
        PolynomialFeatures(degree=spline_order, include_bias=False),
        Ridge(alpha=0.01,solver='cholesky',normalize=True)
        )
    pred_polyreg.fit(flat_pred_col, flat_pred_row)
    #print('new label shape : {} new pred shape : {}'.format(new_label_x.shape, new_pred_x.shape))
    new_label_y = label_polyreg.predict(new_label_x.reshape(-1,1))
    new_pred_y = pred_polyreg.predict(new_pred_x.reshape(-1,1))
    new_label_y = new_label_y[:,0]
    new_pred_y = new_pred_y[:,0]
    #print('new label shape : {} new pred shape : {}'.format(new_label_y, new_pred_y))
    ########    ########    ########    ########    ########    ########
    
    h, w = np.shape(ori_img)[0], np.shape(ori_img)[1]

    new_int_label_x = new_label_x.astype(np.uint16)
    new_int_label_y = np.around(new_label_y).astype(np.uint16)
    #print('int label : {}'.format(new_int_label_y))
    
    new_int_pred_x = new_pred_x.astype(np.uint16)
    new_int_pred_y = np.around(new_pred_y).astype(np.uint16)
    
    temp_poly = ori_img.copy()
    temp_mask_poly = np.zeros((shape_img[0], shape_img[1], 3), dtype=np.uint16)
    temp_poly[new_int_pred_y, new_int_pred_x] = (0,255,0)
    temp_mask_poly[new_int_pred_y, new_int_pred_x] = (0,255,0)
    
    
    temp_label_poly = np.zeros((shape_img[0], shape_img[1], 3), dtype=np.uint16)
    temp_label_poly[new_int_label_y, new_int_label_x] = (0,255,0)
 
    
    if len(new_label_x) > len(new_pred_x): # new pred x에 맞춰 진행 (작은쪽))
        min_idx = np.where(np.min(np.floor(new_pred_x)) == np.floor(new_label_x))
        if min_idx[0].size == 0:
            min_idx = np.where(np.floor(new_pred_x) == np.min(np.floor(new_label_x)))
            
        max_idx = np.where(np.max(np.floor(new_pred_x)) == np.floor(new_label_x))
        if max_idx[0].size == 0:
            max_idx = np.where(np.floor(new_pred_x) == np.max(np.floor(new_label_x)))
        #print('min idx : {} max idx : {}'.format(min_idx, max_idx))
        #print('min idx : {} max idx : {}'.format(min_idx[0][0], max_idx[0][0]))
        
        mean_diff = np.zeros(int(max_idx[0][0]) - int(min_idx[0][0]) + 1,dtype=np.float32)
        mean_ym_diff = np.zeros(int(max_idx[0][0]) - int(min_idx[0][0]) + 1,dtype=np.float32)
        
        pred_idx = 0
        for label_idx in range(min_idx[0][0] , max_idx[0][0]):
            mean_diff[pred_idx] = np.abs(new_label_y[label_idx] - new_pred_y[pred_idx])
            mean_ym_diff[pred_idx] = np.abs(new_label_y[label_idx] - new_pred_y[pred_idx]) * _y_convert
            #print(' label x : {} '.format(new_label_x[label_idx]))
            #print(' pred x : {} '.format(new_pred_x[pred_idx]))
            pred_idx = pred_idx +1
        
        
    else:
        min_idx = np.where(np.min(np.floor(new_label_x)) == np.floor(new_pred_x))
        if min_idx[0].size == 0:
            min_idx = np.where(np.floor(new_label_x) == np.min(np.floor(new_pred_x)))
        max_idx = np.where(np.max(np.floor(new_label_x)) == np.floor(new_pred_x))
        if max_idx[0].size ==0:
            max_idx = np.where(np.floor(new_label_x) == np.max(np.floor(new_pred_x)))
        #print('min idx : {} max idx : {}'.format(min_idx, max_idx))
        #print('min idx : {} max idx : {}'.format(min_idx[0][0], max_idx[0][0]))
        mean_diff = np.zeros(int(max_idx[0][0]) - int(min_idx[0][0]) + 1,dtype=np.float32)
        mean_ym_diff = np.zeros(int(max_idx[0][0]) - int(min_idx[0][0]) + 1,dtype=np.float32)
        
        
        label_idx = 0
        for pred_idx in range(min_idx[0][0] , max_idx[0][0]):
            mean_diff[label_idx] = np.abs(new_label_y[label_idx] - new_pred_y[pred_idx])
            mean_ym_diff[label_idx] = np.abs(new_label_y[label_idx] - new_pred_y[pred_idx]) * _y_convert
            #print(' label x : {} '.format(new_label_x[label_idx]))
            #print(' pred x : {} '.format(new_pred_x[pred_idx]))
            label_idx = label_idx+1
    
    #plt.plot(label_col, label_row, "o", new_label_x, new_label_y)
    #plt.plot(pred_col, pred_row, "g", new_pred_x, new_pred_y)
    
    if debug_flag == True:
        #plt.plot(new_label_x, new_label_y, "r")
        plt.plot(new_pred_x, new_pred_y ,"g")

        plt.show()
    
    return np.mean(mean_diff),np.mean(mean_ym_diff), temp_poly, temp_mask_poly,temp_label_poly, label_polyreg, pred_polyreg
    
    
def prediction_reports(label_img, pred_img,shape_img,ori_img,save_folder):
    
    reports_label_img = np.zeros((shape_img[0], shape_img[1]))
    reports_pred_img = np.zeros((shape_img[0], shape_img[1]))
    temp_ori_img = ori_img.copy()
    
    
    label_row, label_col = np.where(np.all(label_img == (255,0,0),axis=-1))
    pred_row, pred_col = np.nonzero(pred_img[:,:,0])
    cv2.imwrite(save_folder + 'raw_img.png', temp_ori_img)
    temp_ori_img[pred_row, pred_col] = (0,0,255)
    cv2.imwrite(save_folder + 'raw_pred_img.png', temp_ori_img)
    temp_ori_img[label_row, label_col] = (255,0,0)
    
    #print('row : {} col : {}'.format(label_row, label_col))
    concat_label_pred_row = np.concatenate((label_row, pred_row),axis = 0)
    concat_label_pred_col = np.concatenate((label_col, pred_col),axis = 0)

    max_concat_row = np.max(concat_label_pred_row)
    min_concat_row = np.min(concat_label_pred_row)
    max_concat_col = np.max(concat_label_pred_col)
    min_concat_col = np.min(concat_label_pred_col)
    
    norm_label_row = np.array((label_row - min_concat_row) / (max_concat_row - min_concat_row),dtype=np.float32 )
    norm_pred_row = np.array((pred_row - min_concat_row) / (max_concat_row - min_concat_row), dtype=np.float32)
    norm_label_col = np.array((label_row - min_concat_col) / (max_concat_col - min_concat_col),dtype=np.float32 )
    norm_pred_col = np.array((pred_row - min_concat_col) / (max_concat_col - min_concat_col), dtype=np.float32)
    
    
    norm_label_idx = zip(norm_label_row, norm_label_col)
    norm_pred_idx = zip(norm_pred_row, norm_pred_col)
    
    label_idx = zip(label_row, label_col)
    cv2.imwrite(save_folder + 'ground_img.png', temp_ori_img)
    pred_idx = zip(pred_row, pred_col)
    
    label_coord= [z for z in label_idx]
    pred_coord = [z for z in pred_idx]
    norm_label_coord = [z for z in norm_label_idx]
    norm_pred_coord = [z for z in norm_pred_idx]
    
    
    distance_metric_array = get_distance_metric(label_coord, pred_coord, norm_label_coord, norm_pred_coord)
    
    
    reports_label_img[label_row,label_col] = 1
    reports_pred_img[pred_row, pred_col] = 1
        
    overlap_region = cv2.bitwise_and(reports_pred_img,reports_label_img)
    temp_row, temp_col = np.nonzero(overlap_region)
    

    temp_ori_img[temp_row, temp_col] = (0,255,0)
    cv2.imwrite(save_folder + 'different_img.png', temp_ori_img)
     
    overlap_region_cnt = len(temp_row)
    
    dice_coeff = (2* overlap_region_cnt) / (len(label_row) + len(pred_row))
    jaccard_dist = overlap_region_cnt / (len(label_row) + len(pred_row) -overlap_region_cnt )
    
    
    tp_cnt =  overlap_region_cnt
    fp_cnt =  (len(label_row) + len(pred_row) - overlap_region_cnt) - len(label_row)
    fn_cnt = (len(label_row) + len(pred_row) - overlap_region_cnt) - len(pred_row)
    return dice_coeff, jaccard_dist, distance_metric_array
    
def get_distance_metric(label, pred,norm_label, norm_pred):
    _x_convert = 12.5000
    _y_convert = 3.9216
    
    from scipy.spatial.distance import directed_hausdorff
    import similaritymeasures
    distance_metric_array = np.zeros(7, dtype=np.float32)
    
    haus_dist = directed_hausdorff(label, pred)[0]
    haus_idx = directed_hausdorff(label, pred)[1:]
    
    print('haus idx : ', haus_idx)
    haus_label_coord = label[haus_idx[0]]
    haus_pred_coord = pred[haus_idx[1]]
    print('haus label coord : {}, haus pred coord : {}'.format(haus_label_coord, haus_pred_coord))
    
    ym_haus_dist = np.sqrt(np.square((haus_label_coord[0] - haus_pred_coord[0]) * _y_convert) + np.square((haus_label_coord[1] - haus_pred_coord[1]) * _x_convert) )
    print(ym_haus_dist)
    
    frechet_dist = similaritymeasures.frechet_dist(label, pred)
    dtw, d = similaritymeasures.dtw(label, pred)
    
    norm_haus_dist = directed_hausdorff(norm_label, norm_pred)[0]
    norm_frechet_dist = similaritymeasures.frechet_dist(norm_label, norm_pred)
    norm_dtw, norm_d = similaritymeasures.dtw(norm_label, norm_pred)
    
    distance_metric_array[0] = haus_dist
    distance_metric_array[1] = frechet_dist
    distance_metric_array[2] = dtw
    distance_metric_array[3] = norm_haus_dist
    distance_metric_array[4] = norm_frechet_dist
    distance_metric_array[5] = norm_dtw
    distance_metric_array[6] = ym_haus_dist
    
    return distance_metric_array
    
def test_diff_spline_fitting_reconNori_img(recon_img,shape_img,spline_order, ori_img, debug_flag):
    '''
    [TO-DO]
        - visualization spline fitting result (ori <-> recon)
        - Blend-altman plot
    '''
    
    from scipy.interpolate import interp1d
    import matplotlib.pyplot as plt
    from scipy.optimize import curve_fit
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.linear_model import LinearRegression
    from sklearn.pipeline import make_pipeline
    from sklearn.linear_model import Ridge

    _x_convert = 12.5000
    _y_convert = 3.9216
    
    temp_img = np.zeros((shape_img[0], shape_img[1], 3))
    
    #temp_recon = np.zeros((shape_img[0], shape_img[1], 3), dtype=np.float32)
    
    pred_row, pred_col = np.nonzero(recon_img[:,:,0])
    temp_img[pred_row, pred_col , : ] = (0,255,0)
    temp_img = temp_img.astype(np.uint8)
    
    if debug_flag == True:
        show_on_jupyter(temp_img)
    temp_ori = ori_img.copy()
    temp_ori[pred_row, pred_col , : ] = (0,255,0)
    if debug_flag == True:
        show_on_jupyter(temp_ori)

    
    flat_pred_col = pred_col.reshape(-1,1)
    flat_pred_row = pred_row.reshape(-1,1)
    
    new_pred_x = np.linspace(np.min(pred_col), np.max(pred_col), int(np.max(pred_col) - np.min(pred_col)+1))
    
    
    #### sklearn poly fit experiment, (For apply the regularization term)
    
    pred_polyreg = make_pipeline(
        PolynomialFeatures(degree=spline_order, include_bias=False),
        Ridge(alpha=0.005,solver='cholesky',normalize=True)
        )
   
    pred_polyreg.fit(flat_pred_col, flat_pred_row)
    
    #print('new label shape : {} new pred shape : {}'.format(new_label_x.shape, new_pred_x.shape))
    new_pred_y = pred_polyreg.predict(new_pred_x.reshape(-1,1))
    new_pred_y = new_pred_y[:,0]
    #print('new label shape : {} new pred shape : {}'.format(new_label_y, new_pred_y))
    ########    ########    ########    ########    ########    ########
    
    h, w = np.shape(ori_img)[0], np.shape(ori_img)[1]
    
    new_int_pred_x = new_pred_x.astype(np.uint16)
    new_int_pred_y = np.around(new_pred_y).astype(np.uint16)
    
    temp_poly = ori_img.copy()
    temp_mask_poly = np.zeros((shape_img[0], shape_img[1], 3), dtype=np.uint16)
    temp_poly[new_int_pred_y, new_int_pred_x] = (0,255,0)
    temp_mask_poly[new_int_pred_y, new_int_pred_x] = (0,255,0)
    
    
    if debug_flag == True:
        #plt.plot(new_label_x, new_label_y, "r")
        plt.plot(new_pred_x, new_pred_y ,"g")

        plt.show()
    
    return temp_poly, temp_mask_poly, pred_polyreg

In [ ]:
name_experiment = 'final_paper_debug_noReg/'
dir_checker('./pipe_result/'+name_experiment)

In [ ]:
# Hyperparam
debug_flag = True
save_folder = './pipe_result/'+str(name_experiment)+'/lc_processing/'
dir_checker(save_folder)
stride_gap = 10

save_folder2 = './pipe_result/'+str(name_experiment)+'/lc_reconstruction/'
dir_checker(save_folder2)

spline_order = 30
curve_fit_method =  'non_linear'
connect_thr_val = 50
save_folder3 = './pipe_result/'+str(name_experiment)+'/post_proc/'
dir_checker(save_folder3)

real_copy_ori_img = RESULT_IMGS.copy()
# processing
all_mean_diff = []
all_mean_ym_diff = []
unknown_cnt =0 

# prediction reports
mean_dice = []; mean_jaccard = []
mean_haus = []; mean_frechet = []; mean_dtw = []
mean_norm_haus = []; mean_norm_frechet = []; mean_norm_dtw = []
mean_ym_haus = []

label_coef_lists = []
pred_coef_lists = []


LC_POLY_GT = np.zeros(((NUM_YOLO_RESULT,500, 760, 3)),dtype=np.uint8)

for idx in range(NUM_YOLO_RESULT):
#for idx in range(10):
    save_folder = './pipe_result/'+str(name_experiment)+'/lc_processing/' + str(idx)+'/'
    save_folder2 = './pipe_result/'+str(name_experiment)+'/lc_reconstruction/' +str(idx)+ '/'
    dir_checker(save_folder)
    dir_checker(save_folder2)
    
    lc_xmin = int(YOLO_RESULT_TABLE.iloc[idx]['lc_xmin'])
    lc_ymin = int(YOLO_RESULT_TABLE.iloc[idx]['lc_ymin'])
    lc_xmax = int(YOLO_RESULT_TABLE.iloc[idx]['lc_xmax'])
    lc_ymax = int(YOLO_RESULT_TABLE.iloc[idx]['lc_ymax'])
    lc_bbox_coord = (lc_xmin,lc_ymin, lc_xmax, lc_ymax)
    temp_ori_img = ORI_IMGS[idx]
    
    # pred_lc_class02 -> lc prediction
    pred_lc_class01, pred_lc_class02,pred_lc_thr_class01,pred_lc_thr_class02,new_h, new_w = lc_overlap_tile_prediction(temp_ori_img, model,
                                                                lc_bbox_coord,stride_gap, 
                                                              NUM_YOLO_RESULT,INFERENCE_SHAPE,
                                                              save_folder, debug_flag)
    
    avg_recon_lc = lc_reconstruction_avg(pred_lc_class02, INFERENCE_SHAPE, new_h, new_w,stride_gap,
                                        'original',save_folder2, debug_flag)
    
    avg_recon_thr_lc = lc_reconstruction_avg(pred_lc_thr_class02, INFERENCE_SHAPE, new_h, new_w,stride_gap,
                                        'threshold',save_folder2 ,debug_flag)
        
    prev_h = int(lc_ymax - lc_ymin); prev_w = int(lc_xmax - lc_xmin)
    diff_h = new_h - prev_h; diff_w = new_w - prev_w
    recon_pred_img = pred_reconstruction_oct(temp_ori_img, avg_recon_lc, lc_bbox_coord, 
                            diff_h, diff_w, save_folder2, debug_flag)
    
    post_recon_img = post_proc_recon_img(recon_pred_img, SHAPE_YOLO_RESULT, connect_thr_val,idx, save_folder3, debug_flag)
    temp_check_row , temp_check_col = np.nonzero(post_recon_img[:,:,0])
    if len(temp_check_row) !=0:
        if VALID_TEST == 'valid':
            mean_diff,mean_ym_diff, RESULT_IMGS[idx], RESULT_IMGS_MASKS[idx],LC_POLY_GT[idx], temp_label_coef, temp_pred_coef = diff_spline_fitting_reconNori_img(GROUND_TRUTH_IMG[idx], post_recon_img, SHAPE_YOLO_RESULT,spline_order, ORI_IMGS[idx], debug_flag)
            print('label coef : ', temp_label_coef)
            print('pref coef : ', temp_pred_coef)
            label_coef_lists.append(temp_label_coef); pred_coef_lists.append(temp_pred_coef)

            all_mean_diff.append(mean_diff)
            all_mean_ym_diff.append(mean_ym_diff)
            dice_score, jaccard, dist_array = prediction_reports(GROUND_TRUTH_IMG[idx], post_recon_img,SHAPE_YOLO_RESULT,ORI_IMGS[idx],save_folder2)
            mean_dice.append(dice_score); mean_jaccard.append(jaccard)
            mean_haus.append(dist_array[0]); mean_frechet.append(dist_array[1]); mean_dtw.append(dist_array[2])
            mean_norm_haus.append(dist_array[3]); mean_norm_frechet.append(dist_array[4])
            mean_norm_dtw.append(dist_array[5])
            mean_ym_haus.append(dist_array[6])
        elif VALID_TEST == 'test':
            RESULT_IMGS[idx], RESULT_IMGS_MASKS[idx],temp_pred_coef = test_diff_spline_fitting_reconNori_img(post_recon_img, SHAPE_YOLO_RESULT,spline_order, ORI_IMGS[idx], debug_flag)
            pred_coef_lists.append(temp_pred_coef)
    else:
        unknown_cnt = unknown_cnt+1
        pass
    
    #spline_fitting_recon_img(avg_recon_thr_lc,5)
    #post_proc_recon_img(avg_recon_lc,0.4)
    
    

In [ ]:
segmentation_table = pd.DataFrame(columns= ['Mean dice', 'Mean jaccard', 'Mean average y-diff', 'Std average y-diff'])

curve_sim_table = pd.DataFrame(columns=['Mean haus dist','Std haus dist', 'Mean frechet dist','Std frechet dist', 'Mean DTW', 'Std DTW'])
curve_sim_norm_table = pd.DataFrame(columns=['Mean norm haus dist','Std norm haus dist', 'Mean norm frechet dist','Std norm frechet dist', 'Mean norm DTW','Std norm DTW'])

ym_result_table = pd.DataFrame(columns=['Mean ym average y-diff', 'Std ym average y-diff', 'Mean ym haus dist', 'Std ym haus dist'])

segmentation_table.loc[0,'Mean dice'] = np.mean(mean_dice) ; segmentation_table.loc[0,'Mean jaccard'] = np.mean(mean_jaccard)
segmentation_table.loc[0,'Mean average y-diff'] = np.mean(all_mean_diff); segmentation_table.loc[0,'Std average y-diff'] = np.std(all_mean_diff)

curve_sim_table.loc[0, 'Mean hasu dist'] = np.mean(mean_haus); curve_sim_table.loc[0, 'Std hasu dist'] = np.std(mean_haus);
curve_sim_table.loc[0, 'Mean frechet dist'] = np.mean(mean_frechet); curve_sim_table.loc[0, 'Std frechet dist'] = np.std(mean_frechet);
curve_sim_table.loc[0, 'Mean DTW'] = np.mean(mean_dtw); curve_sim_table.loc[0, 'Std DTW'] = np.std(mean_dtw);

curve_sim_norm_table.loc[0, 'Mean norm hasu dist'] = np.mean(mean_norm_haus); curve_sim_table.loc[0, 'Std norm hasu dist'] = np.std(mean_norm_haus);
curve_sim_norm_table.loc[0, 'Mean norm frechet dist'] = np.mean(mean_norm_frechet); curve_sim_table.loc[0, 'Std norm frechet dist'] = np.std(mean_norm_frechet);
curve_sim_norm_table.loc[0, 'Mean norm DTW'] = np.mean(mean_norm_dtw); curve_sim_table.loc[0, 'Std norm DTW'] = np.std(mean_norm_dtw);


#[TODO] mean ym diff -> all mean ym diff
ym_result_table.loc[0,'Mean ym average y-diff'] =  np.mean(all_mean_ym_diff); ym_result_table.loc[0,'Std ym average y-diff'] =  np.std(all_mean_ym_diff)
ym_result_table.loc[0,'Mean ym haus dist'] =  np.mean(mean_ym_haus); ym_result_table.loc[0,'Std ym haus dist'] =  np.std(mean_ym_haus)

In [ ]:
save_folder = './pipe_result/'+str(name_experiment)

fig,ax = render_mpl_table(segmentation_table, header_columns=0, col_width=4.0)
fig.savefig(save_folder +'/'+str(spline_order) +'_segmentation_table.png')

fig,ax = render_mpl_table(curve_sim_table, header_columns=0, col_width=4.0)
fig.savefig(save_folder +'/'+str(spline_order) + '_curve_sim_table.png')

fig,ax = render_mpl_table(curve_sim_norm_table, header_columns=0, col_width=4.0)
fig.savefig(save_folder +'/'+str(spline_order) + '_norm_curve_sim_table.png')

fig,ax = render_mpl_table(ym_result_table, header_columns=0, col_width=4.0)
fig.savefig(save_folder +'/'+str(spline_order) + '_ym_result_table.png')

In [ ]:
print('dice score : ', np.mean(mean_dice))
print('jaccard score : ', np.mean(mean_jaccard))
print('all mean diff : ',np.mean(all_mean_diff))
print('all std diff : ',np.std(all_mean_diff))

print('mean hausdorff distance : ', np.mean(mean_haus))
print('mean frechet distance  : ', np.mean(mean_frechet))
print('mean dynamic time warping  : ',np.mean(mean_dtw))

print('mean normalized hausdorff distance : ', np.mean(mean_norm_haus))
print('mean normalized frechet distance  : ', np.mean(mean_norm_frechet))
print('mean normalized dynamic time warping  : ',np.mean(mean_norm_dtw))

print('std hausdorff distance : ', np.std(mean_haus))
print('std frechet distance  : ', np.std(mean_frechet))
print('std dynamic time warping  : ',np.std(mean_dtw))

print('std normalized hausdorff distance : ', np.std(mean_norm_haus))
print('std normalized frechet distance  : ', np.std(mean_norm_frechet))
print('std normalized dynamic time warping  : ',np.std(mean_norm_dtw))

print('mean ym average y-diff : ', np.mean(mean_ym_diff))
print('std ym average y-diff : ', np.std(mean_ym_diff))
print('mean ym hausdorff distance : ', np.mean(mean_ym_haus))
print('std ym hausdorff distance : ', np.std(mean_ym_haus))


print('unknown cnt : ', unknown_cnt)



In [ ]:
def _neighbors_conv(image):
    import scipy.ndimage as ndi

    """
    Counts the neighbor pixels for each pixel of an image:
            x = [
                [0, 1, 0],
                [1, 1, 1],
                [0, 1, 0]
            ]
            _neighbors(x)
            [
                [0, 3, 0],
                [3, 4, 3],
                [0, 3, 0]
            ]
    :type image: numpy.ndarray
    :param image: A two-or-three dimensional image
    :return: neighbor pixels for each pixel of an image
    """
    image = image.astype(np.int)
    k = np.array([[1,1,1],[1,0,1],[1,1,1]])
    neighborhood_count = ndi.convolve(image,k, mode='constant', cval=1)
    neighborhood_count[~image.astype(np.bool)] = 0
    return neighborhood_count

def get_slope(first_point, second_point):
    (x1, y1) = (first_point[1],first_point[0])
    (x2, y2) = (second_point[1],second_point[0])
    slope = ((y2-y1)/(x2-x1))
    return slope

def get_intersection_perpendicular_n_line(line_p1, line_p2, per_above_points):
    '''
    line : y = ax + b
    perpen line : y = (-1/a)x + c
    
    1. we know the point of above to perpendicular line -> (x_p, y_p)
    2. y_p = (-1/a)x_p + c -> can find the 'c'
    3. Example) y = x + 3 / y = -x -7 (a,b) -> intersectin
       b = a + 3 / b = -a - 7
    4. ax + b = -(1/a)x + c
       ax + (1/a)x= c - b
       (a + 1/a)x = c - b
       x = (c - b) / (a + 1/a)
    
    '''
    line_slope = (line_p1[0] - line_p2[0]) / (line_p1[1] - line_p2[1])
    line_constant = line_p1[0] - (line_slope * line_p1[1])
    
    perpen_slope = -1/line_slope
    perpen_constant = per_above_points[0] - (perpen_slope * per_above_points[1])
    
    xi = (perpen_constant - line_constant) / (line_slope + (1/line_slope))
    yi = (line_slope * xi) + line_constant
    
    return np.array((yi, xi), dtype=np.uint16)
    
    

def get_perpendicular_dist(line_1_point, line_2_point, target_points_row, target_points_col):
    '''
    code from : https://stackoverflow.com/questions/39840030/distance-between-point-and-a-line-from-two-points/39840218
    line_1_point : 'point not points!' Max or min point
    line_2_point ; Max or min point
    
    target_points_row :
    target_points_col : 
    
    '''
    d_arr = np.zeros(len(target_points_row), dtype=np.float32)
    # row, col
    p1 = np.array((line_1_point[1], line_1_point[0]))
    p2 = np.array((line_2_point[1], line_2_point[0]))
    print('p1 , p2 : {} {} ', p1, p2)
    
    '''
    [TODO]
        curve depth -> check the their depth (+ or -)
        
    '''
    for idx in range(len(target_points_row)):
        p3 = np.array((target_points_col[idx], target_points_row[idx]))
        #
        d = np.linalg.norm(np.cross(p2-p1, p1-p3))/np.linalg.norm(p2-p1)
        #print('cross product : ',np.cross(p2-p1, p1-p3))
        if np.cross(p2-p1, p1-p3) >= 0:
            d = -d
        d_arr[idx] = d
        #print(d)
    
    max_d = np.max(d_arr)
    max_d_idx = np.where(d_arr == np.max(d_arr))
    print('max_d : ',max_d)
    print('max d idx : ', max_d_idx)
    #print('max coord row : {} col : {} '.format(target_points_row[max_d_idx], target_points_col[max_d_idx]))
    max_coord = np.array((target_points_row[max_d_idx], target_points_col[max_d_idx]))
    print('max coord ',max_coord)
    #return max_d, max_coord
    if len(max_coord[0]) ==1:
        print('max coord len == 1 ')
        return max_d, max_coord
    else:
        print('max coord len > 1 ')
        print('max coord[0][0] [0][1] ',target_points_row[max_d_idx][0],target_points_col[max_d_idx][0])
        return max_d, (target_points_row[max_d_idx][0],target_points_col[max_d_idx][0])
    
#### draw 


def createLineIterator(P1, P2, img, flag = False):
    """
    Produces and array that consists of the coordinates and intensities of each pixel in a line between two points

    Parameters:
        -P1: a numpy array that consists of the coordinate of the first point (x,y)
        -P2: a numpy array that consists of the coordinate of the second point (x,y)
        -img: the image being processed

    Returns:
        -it: a numpy array that consists of the coordinates and intensities of each pixel in the radii (shape: [numPixels, 3], row = [x,y,intensity])     
    """
    #define local variables for readability
    imageH = img.shape[0]
    imageW = img.shape[1]
    P1X = P1[0]
    P1Y = P1[1]
    P2X = P2[0]
    P2Y = P2[1]

    #difference and absolute difference between points
    #used to calculate slope and relative location between points
    dX = P2X - P1X
    dY = P2Y - P1Y
    dXa = np.abs(dX)
    dYa = np.abs(dY)

    #predefine numpy array for output based on distance between points
    itbuffer = np.empty(shape=(np.maximum(dYa,dXa),3),dtype=np.float32)
    itbuffer.fill(np.nan)

    #Obtain coordinates along the line using a form of Bresenham's algorithm
    negY = P1Y > P2Y
    negX = P1X > P2X
    if P1X == P2X: #vertical line segment
        itbuffer[:,0] = P1X
        if negY:
            itbuffer[:,1] = np.arange(P1Y - 1,P1Y - dYa - 1,-1)
        else:
            itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)              
    elif P1Y == P2Y: #horizontal line segment
        itbuffer[:,1] = P1Y
        if negX:
            itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
        else:
            itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
    else: #diagonal line segment
        steepSlope = dYa > dXa
        if steepSlope:
            slope = dX.astype(np.float32)/dY.astype(np.float32)
            #print('slope : ' , slope)
            if negY:
                itbuffer[:,1] = np.arange(P1Y-1,P1Y-dYa-1,-1)
            else:
                itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)
            #print(itbuffer)
            itbuffer[:,0] = (slope*(itbuffer[:,1]-P1Y)).astype(np.int) + P1X
        else:
            slope = dY.astype(np.float32)/dX.astype(np.float32)
            #print('slope : ' , slope)
            if negX:
                itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
            else:
                itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
            #print(itbuffer)
            itbuffer[:,1] = (slope*(itbuffer[:,0]-P1X)).astype(np.int) + P1Y

   #Remove points outside of image
    colX = itbuffer[:,0]
    colY = itbuffer[:,1]
    #print('colY : ',colY)
    #print('max colY :', max(colY))
    if flag == 'max':
        max_idx = np.where(colY == int(max(colY)))
        #print('max : ',max_idx[0][0])
        max_idx = max_idx[0][0]
        #print('colX : ',colX)
        #print('colY : ',colY)
        return itbuffer, (colX[max_idx], colY[max_idx])
    elif flag == 'min':
        min_idx = np.where(colY == int(min(colY)))
        #print('max : ',max_idx[0][0])
        min_idx = min_idx[0][0]
        #print('colX : ',colX)
        #print('colY : ',colY)
        return itbuffer, (colX[min_idx], colY[min_idx])
    #print(itbuffer)
    
    #itbuffer = itbuffer[(colX >= 0) & (colY >=0) & (colX<imageW) & (colY<imageH)]
    #print(itbuffer)
    
    #Get intensities from img ndarray
    #itbuffer[:,2] = img[itbuffer[:,1].astype(np.uint16),itbuffer[:,0].astype(np.uint16)]

    return itbuffer


def getDashLine(line, dash_constant = 3, delete_connect_val = 1):
    delete_idx = []
    for idx in range(len(line)):
        if idx % dash_constant ==0:
            delete_idx.append(idx)
            for con_idx in range(delete_connect_val-1):
                delete_idx.append(idx + con_idx)
                
    dash_line = np.delete(line, delete_idx, axis =0 )
    return dash_line
    

def getSlopeVal(point1, point2):
    '''
    point1, point2 = (height, width) = (y, x)
    
    '''
    slope = (point1[0] - point2[0]) / (point1[1] - point2[1])
    return slope

def getPerpendicularLineList(point, slope, target_y = 500):
    '''
    - 길이를 정하면 될 듯
    point[0] = h(y) , point[1] = w(x)
    
    '''    
    import numpy as np
    
    if slope != 0 :
        perpendicular_slope = -(1/slope)
    else:
        slope = 1e-12
        perpendicular_slope = -(1/slope)
    # y = ax+b / y - ax -b =0
    a = perpendicular_slope
    b = point[0] - (perpendicular_slope * point[1])
    #print('a : ',a)
    #print('b : ',b)
    target_x = (target_y - b)/a
    
    #print(target_y, target_x)
    
    return np.asarray((int(target_y), int(target_x))), b
    
def lineIterToCoord(line_iter_result):
    coord_list = []
    for i in range(len(line_iter_result)):
        h = int(line_iter_result[i][0])
        w = int(line_iter_result[i][1])
        coord_list.append(np.asarray((h,w)))
        
    return np.asarray(coord_list)

def lcMergeCoord(lc_row, lc_col):
    coord_list = []
    for i in range(len(lc_row)):
        coord_list.append(np.asarray((lc_row[i], lc_col[i])))
        
    return np.asarray(coord_list)

def left_right_lc_neighbors_conv(image,mode):
    import scipy.ndimage as ndi

    """
    Counts the neighbor pixels for each pixel of an image:
            x = [
                [0, 1, 0],
                [1, 1, 1],
                [0, 1, 0]
            ]
            _neighbors(x)
            [
                [0, 3, 0],
                [3, 4, 3],
                [0, 3, 0]
            ]
    :type image: numpy.ndarray
    :param image: A two-or-three dimensional image
    :return: neighbor pixels for each pixel of an image
    """
    if mode == 'right':
        image = image.astype(np.int)
        k = np.array([[1,1,0],[1,0,0],[1,1,0]])
        neighborhood_count = ndi.convolve(image,k, mode='constant', cval=1)
        neighborhood_count[~image.astype(np.bool)] = 0
        return neighborhood_count
    elif mode =='left':
        image = image.astype(np.int)
        k = np.array([[0,1,1],[0,0,1],[0,1,1]])
        neighborhood_count = ndi.convolve(image,k, mode='constant', cval=1)
        neighborhood_count[~image.astype(np.bool)] = 0
        return neighborhood_count

def getWidthPoint(img_shape, lc_info, L_perpen_line_info, R_perpen_line_info, debug_mode =True):
    '''
    img_shape : (num, height, width, ch)
    
    '''
    zero_img = np.zeros((img_shape[1], img_shape[2]), dtype=np.uint16)
    zero_3ch_img = np.zeros((img_shape[1], img_shape[2],3), dtype=np.uint16)
    
    zero_img[lc_info] = True
    
    mask_neighbors = (_neighbors_conv(zero_img) == 1)
    temp_row, temp_col = np.where(mask_neighbors == True)

    min_row_idx = np.where(temp_row == np.min(temp_row))
    min_col_idx = np.where(temp_col == np.min(temp_col))
    max_row_idx = np.where(temp_row == np.max(temp_row))
    max_col_idx = np.where(temp_col == np.max(temp_col))

    min_end_coord =np.array((temp_row[min_col_idx[0][0]] ,temp_col[min_col_idx[0][0]]))
    max_end_coord =np.array((temp_row[max_col_idx[0][0]] ,temp_col[max_col_idx[0][0]]))
    zero_img[L_perpen_line_info] = True
    left_width_candi_point = left_right_lc_neighbors_conv(zero_img, 'left')
    if np.max(left_width_candi_point) <3: #right...
        left_width_point = min_end_coord
        print('[1] left type : {} left shape : {}'.format(type(left_width_point), np.shape(left_width_point)))

    else:
        left_width_point = np.where(left_width_candi_point == np.max(left_width_candi_point))
        print('[2] left type : {} left shape : {}'.format(type(left_width_point), np.shape(left_width_point)))

        if np.shape(left_width_point )[1] >=2:
            left_width_point = np.array((left_width_point[0][0], left_width_point[1][0]))
            print('[3] left type : {} left shape : {}'.format(type(left_width_point), np.shape(left_width_point)))
        else:
            left_width_point = np.array((left_width_point[0][0], left_width_point[1][0]))
    
    zero_img[L_perpen_line_info] = False
    zero_img[R_perpen_line_info] = True
    right_width_candi_point = left_right_lc_neighbors_conv(zero_img, 'right')
    
    if np.max(right_width_candi_point) < 3:
        right_width_point = max_end_coord
    else:
        right_width_point = np.where(right_width_candi_point == np.max(right_width_candi_point))
        if np.shape(right_width_point)[1]  >=2:
            right_width_point = np.array((right_width_point[0][0], right_width_point[1][0]))
            #right_width_point = [right_width_point[0][0], right_width_point[1][0]]
        else:
            right_width_point = np.array((right_width_point[0][0], right_width_point[1][0]))
    
    if debug_mode ==True:
        zero_3ch_img[lc_info[0], lc_info[1],:] = (255,255,0)
        zero_3ch_img[L_perpen_line_info[0],L_perpen_line_info[1] ,:] = (255,255,255)
        zero_3ch_img[R_perpen_line_info[0], R_perpen_line_info[1],:] = (255,255,255)
        
        print('left : ',left_width_point)
        print('right : ',right_width_point)
        cv2.circle(zero_3ch_img ,(left_width_point[1], left_width_point[0]), 3,(0,255,255))
        cv2.circle(zero_3ch_img ,(right_width_point[1], right_width_point[0]), 3,(0,255,255))
        
        show_on_jupyter(zero_3ch_img, fig_size=(15,15))

        
    return np.asarray(left_width_point), np.asarray(right_width_point)

def detectPeakPoint(lc_row, lc_col):
    from scipy.signal import find_peaks
    lc_col, lc_row = zip(*sorted(zip(lc_col, lc_row)))
    peaks, _ = find_peaks(lc_row, distance=3)
    print('shape : {}'.format(np.shape(peaks)))
    
    max_idx =0
    prev_row = 0
    curr_row = 0 
    for idx in range(np.shape(peaks)[0]):
        prev_row = curr_row
        ext_idx = peaks[idx]
        ext_row, ext_col = lc_row[ext_idx], lc_col[ext_idx]
        curr_row = ext_row
        if curr_row > prev_row:
            max_idx = ext_idx
            
    return lc_row[max_idx], lc_col[max_idx]
        
def getBmoPerpendicularDist(line_1_point, line_2_point, target_points_row, target_points_col):
    '''
    code from : https://stackoverflow.com/questions/39840030/distance-between-point-and-a-line-from-two-points/39840218
    line_1_point : 'point not points!' Max or min point
    line_2_point ; Max or min point
    
    target_points_row :
    target_points_col : 
    
    '''
    # row, col
    p1 = np.array((line_1_point[1], line_1_point[0]))
    p2 = np.array((line_2_point[1], line_2_point[0]))
    
    '''
    [TODO]
        curve depth -> check the their depth (+ or -)
        
    '''
    p3 = np.array((target_points_col, target_points_row))
        #
    d = np.linalg.norm(np.cross(p2-p1, p1-p3))/np.linalg.norm(p2-p1)
    return d

def drawLineUsingEndPoint(end_points,slope, draw_len = 50):
    # y = ax + b
    b = -(slope* end_points[0] ) + end_points[1]
    
    #for i in range(draw_len):
        
def physical_distance_converter(ref_points, target_points):
    '''
    
    
    '''
    _x_convert = 12.5000
    _y_convert = 3.9216
    print('ref points : {} target points : {}'.format(ref_points, target_points))
    
    dist = np.sqrt(np.square((ref_points[0] - target_points[0]) * _y_convert, dtype = np.float32) + np.square((ref_points[1] - target_points[1]) * _x_convert, dtype= np.float32) ,dtype= np.float32)
    return dist

### f

In [ ]:
'''
This section is optimized for inha hosital data (2020.11)


'''
save_folder =  '/home/bono/Desktop/mellab_project/data/set1_inhaHos/'+'quant_result/' + file_num +'/'
dir_checker(save_folder)

idx = 15


if idx ==  0:
    filename = '001005'
    save_folder =   '/home/bono/Desktop/mellab_project/data/set1_inhaHos/'+'quant_result/' + file_num +'/'+filename+'/'
elif idx == 1:
    filename = '001006'
    save_folder =   '/home/bono/Desktop/mellab_project/data/set1_inhaHos/'+'quant_result/' + file_num +'/'+filename+'/'
elif idx == 2:
    filename = '001007'
    save_folder =   '/home/bono/Desktop/mellab_project/data/set1_inhaHos/'+'quant_result/' + file_num +'/'+filename+'/'
elif idx == 3:
    filename = '001008'
    save_folder =   '/home/bono/Desktop/mellab_project/data/set1_inhaHos/'+'quant_result/' + file_num +'/'+filename+'/'
elif idx == 4:
    filename = '002005' #
    save_folder =   '/home/bono/Desktop/mellab_project/data/set1_inhaHos/'+'quant_result/' + file_num +'/'+filename+'/'
elif idx == 5:
    filename = '002006'
    save_folder =   '/home/bono/Desktop/mellab_project/data/set1_inhaHos/'+'quant_result/' + file_num +'/'+filename+'/'
elif idx == 6:
    filename = '002007'
    save_folder =   '/home/bono/Desktop/mellab_project/data/set1_inhaHos/'+'quant_result/' + file_num +'/'+filename+'/'
elif idx == 7:
    filename = '002008'
    save_folder =   '/home/bono/Desktop/mellab_project/data/set1_inhaHos/'+'quant_result/' + file_num +'/'+filename+'/'
elif idx == 8:
    filename = '003005'
    save_folder =   '/home/bono/Desktop/mellab_project/data/set1_inhaHos/'+'quant_result/' + file_num +'/'+filename+'/'
elif idx == 9:
    filename = '003006'#
    save_folder =   '/home/bono/Desktop/mellab_project/data/set1_inhaHos/'+'quant_result/' + file_num +'/'+filename+'/'
elif idx == 10:
    filename = '003007'
    save_folder =   '/home/bono/Desktop/mellab_project/data/set1_inhaHos/'+'quant_result/' + file_num +'/'+filename+'/'
elif idx == 11:
    filename = '003008'
    save_folder =   '/home/bono/Desktop/mellab_project/data/set1_inhaHos/'+'quant_result/' + file_num +'/'+filename+'/'
elif idx == 12:
    filename = '004005'
    save_folder =   '/home/bono/Desktop/mellab_project/data/set1_inhaHos/'+'quant_result/' + file_num +'/'+filename+'/'
elif idx == 13:
    filename = '004006'
    save_folder =   '/home/bono/Desktop/mellab_project/data/set1_inhaHos/'+'quant_result/' + file_num +'/'+filename+'/'
elif idx == 14:
    filename = '004007'
    save_folder =   '/home/bono/Desktop/mellab_project/data/set1_inhaHos/'+'quant_result/' + file_num +'/'+filename+'/'
elif idx == 15:
    filename = '004008'
    save_folder =   '/home/bono/Desktop/mellab_project/data/set1_inhaHos/'+'quant_result/' + file_num +'/'+filename+'/'
    
    
dir_checker(save_folder)

_x_convert = 12.5000
_y_convert = 3.9216

lc_width_arr = np.zeros(NUM_YOLO_RESULT, dtype=np.float32)
no_post_lc_width_arr = np.zeros(NUM_YOLO_RESULT, dtype=np.float32)


copy_result_img = RESULT_IMGS.copy()
copy_result_mask = RESULT_IMGS_MASKS.copy()

PRED_LCD = []
PRED_LCCD = []
PRED_LCCI = []

no_post_PRED_LCD = []
no_post_PRED_LCCD = []
no_post_PRED_LCCI = []

dash_constant = 5


## start!!!

temp_left = recon_bmo01_center[idx]
temp_right = recon_bmo02_center[idx]

# change the left & right
if recon_bmo01_center[idx][1] > recon_bmo02_center[idx][1]:
    recon_bmo01_center[idx] = temp_right
    recon_bmo02_center[idx] = temp_left
print('recon bmo center type : ', type(recon_bmo01_center))

cv2.imwrite(save_folder+filename+'_lc_img.png',copy_result_img[idx])
# debugging process
RESULT_IMGS[idx, recon_bmo01_center[idx][0], recon_bmo01_center[idx][1]] = (0,0,255)
RESULT_IMGS[idx, recon_bmo02_center[idx][0], recon_bmo02_center[idx][1]] = (0,0,255) 
copy_result_img[idx, recon_bmo01_center[idx][0], recon_bmo01_center[idx][1]] = (0,0,255)
copy_result_img[idx, recon_bmo02_center[idx][0], recon_bmo02_center[idx][1]] = (0,0,255) 


L_bmo_circle_coord = (recon_bmo01_center[idx][1], recon_bmo01_center[idx][0])
R_bmo_circle_coord = (recon_bmo02_center[idx][1], recon_bmo02_center[idx][0])

cv2.circle(copy_result_img[idx] ,L_bmo_circle_coord, 3,(0,0,255),-1)
cv2.circle(copy_result_img[idx] ,R_bmo_circle_coord, 3,(0,0,255),-1)
show_on_jupyter(copy_result_img[idx], fig_size= (17,17))
cv2.imwrite(save_folder+filename+'_seg.png',copy_result_img[idx])



cv2.line(copy_result_img[idx], (recon_bmo01_center[idx][1], recon_bmo01_center[idx][0]), (recon_bmo02_center[idx][1], recon_bmo02_center[idx][0]), (255,255,0), 2 )
cv2.imwrite(save_folder+filename+'_bmo_ref.png',copy_result_img[idx])

################################################################################################
# perpendicular line to bmo terminate points
line_coord = createLineIterator(np.asarray(recon_bmo01_center[idx]), np.asarray(recon_bmo02_center[idx]), copy_result_img[idx,:,:,0])

slope = getSlopeVal(np.asarray(recon_bmo01_center[idx]), np.asarray(recon_bmo02_center[idx]))
print('slope : ' , slope)
left_target_point,left_b = getPerpendicularLineList(np.asarray(recon_bmo01_center[idx]), slope,target_y=450)
left_per_line_coord, max_left_line_points = createLineIterator(np.asarray(recon_bmo01_center[idx]), left_target_point, copy_result_img[idx,:,:,0], flag = 'min')
left_dash_line_coord = getDashLine(left_per_line_coord, dash_constant=dash_constant, delete_connect_val=3)
copy_result_img[idx,left_dash_line_coord[:,0].astype(np.uint16), left_dash_line_coord[:,1].astype(np.uint16)] = (255,255,255)
#cv2.circle(copy_result_img[idx] ,(recon_bmo01_center[idx][1], recon_bmo01_center[idx][0]), 5,(0,255,255))

right_target_point,right_b = getPerpendicularLineList(np.asarray(recon_bmo02_center[idx]), slope,target_y=450)
right_per_line_coord, max_right_line_points = createLineIterator(np.asarray(recon_bmo02_center[idx]), right_target_point, copy_result_img[idx,:,:,0], flag ='max')
right_dash_line_coord = getDashLine(right_per_line_coord, dash_constant=dash_constant, delete_connect_val=3)
copy_result_img[idx,right_dash_line_coord[:,0].astype(np.uint16), right_dash_line_coord[:,1].astype(np.uint16)] = (255,255,255)

################################################################################################

RESULT_IMGS_MASKS[idx, recon_bmo01_center[idx][0], recon_bmo01_center[idx][1]] = (0,0,255)
RESULT_IMGS_MASKS[idx, recon_bmo02_center[idx][0], recon_bmo02_center[idx][1]] = (0,0,255)

# LC
lc_border_row, lc_border_col = np.where(np.all(RESULT_IMGS[idx] == (0,255,0), axis=-1))
########################## 잠깐 실험...! (polyfit)
print('left line info : {} right line info :{}'.format(max_left_line_points, max_right_line_points))

left_interpol_point = 0
right_interpol_point = 0
if int(max_left_line_points[1]) >  recon_bmo01_center[idx][1]:
    left_interpol_point = recon_bmo01_center[idx][1]
else:
    print('exception! : {}'.format(max_left_line_points[1]))
    left_interpol_point = int(max_left_line_points[1])


if int(max_right_line_points[1]) < recon_bmo02_center[idx][1]:
    right_interpol_point = recon_bmo02_center[idx][1]
else:
    right_interpol_point = int(max_right_line_points[1])

print('right interpol point : ', right_interpol_point)
show_on_jupyter(copy_result_img[idx], fig_size=(17,17))
cv2.imwrite(save_folder+filename+'_lc_interpol.png',copy_result_img[idx])
################################################################################################

L_per_info = (left_per_line_coord[:,0].astype(np.uint16), left_per_line_coord[:,1].astype(np.uint16))
R_per_info = (right_per_line_coord[:,0].astype(np.uint16), right_per_line_coord[:,1].astype(np.uint16))
left, right = getWidthPoint(np.shape(YOLO_IMGS),(lc_border_row,lc_border_col),L_per_info, R_per_info)
print('left : {} right : {}'.format(left, right))
# 흔적

min_end_coord = left
max_end_coord = right
print('min end coord : {} max end coord {} '.format(min_end_coord,max_end_coord))
##################### perpendicular ####################
############################################################
############################################################

left_end_lc_point = left
right_end_lc_point = right
print('right end lc point[1] : {} right interpol point : {}'.format(right_end_lc_point[1], right_interpol_point))
interpol_lc_left_x_range = np.linspace(left_interpol_point, left_end_lc_point[1],int(left_end_lc_point[1] -left_interpol_point+1))
interpol_lc_right_x_range = np.linspace(right_end_lc_point[1], right_interpol_point,int(right_interpol_point-right_end_lc_point[1]+1))
print('idx : ',idx)
print('left interpolation range : {} \n right interpolation range : {}'.format(interpol_lc_left_x_range, interpol_lc_right_x_range))

#print('reshape : {} pred coef[idx] : {} '.format(interpol_lc_left_x_range.reshape(-1,1), pred_coef_lists[idx]))
new_pred_left_lc_y = pred_coef_lists[idx].predict(interpol_lc_left_x_range.reshape(-1,1))
new_pred_left_lc_y = np.around(new_pred_left_lc_y[:,0]).astype(np.uint16)

new_pred_right_lc_y = pred_coef_lists[idx].predict(interpol_lc_right_x_range.reshape(-1,1))
new_pred_right_lc_y = np.around(new_pred_right_lc_y[:,0]).astype(np.uint16)

print('pred cocl shape : {} new pred left lc x shape : {}'.format(lc_border_col.shape, new_pred_left_lc_y.shape))

new_lc_border_col = np.concatenate((interpol_lc_left_x_range, lc_border_col),axis = 0)
new_lc_border_col = np.concatenate((new_lc_border_col, interpol_lc_right_x_range), axis =0)
new_lc_border_col = new_lc_border_col.astype(np.uint16)

new_lc_border_row = np.concatenate((new_pred_left_lc_y, lc_border_row), axis =0 )
new_lc_border_row = np.concatenate((new_lc_border_row, new_pred_right_lc_y), axis =0 )

print('concat row shape : {} concat col shape : {}'.format(new_lc_border_row.shape, new_lc_border_col.shape))

#new_lc_border_row[new_lc_border_row > 500]
#copy_result_img[idx, new_lc_border_row, new_lc_border_col] = (125,125,0)
#show_on_jupyter(copy_result_img[idx], fig_size=(15,15))
print('concat row : {} concat col: {}'.format(new_lc_border_row, new_lc_border_col))
final_lc_left, final_lc_right = getWidthPoint(np.shape(YOLO_IMGS),(new_lc_border_row,new_lc_border_col),L_per_info, R_per_info)

# find final lc point's index
print('final lc left : {}'.format(final_lc_left))
print('row : {} \n\n col : {}'.format(new_lc_border_row, new_lc_border_col))
# delete, 찾은 값 기준 범위
left_col_idx = np.where( new_lc_border_col == final_lc_left[1])
left_row_idx = np.where( new_lc_border_row == final_lc_left[0])
print('final lc left col idx : {} row idx : {}'.format(left_col_idx, left_row_idx))

new_lc_info = np.array((new_lc_border_row, new_lc_border_col))
dist_left_row = new_lc_border_row - final_lc_left[0]
dist_left_col = new_lc_border_col.astype(np.int16) - final_lc_left[1]
dist_left = np.abs(dist_left_row) + np.abs(dist_left_col)
left_idx = (np.where(dist_left == np.min(dist_left)))
left_idx = left_idx[0][0]

dist_right_row = new_lc_border_row - final_lc_right[0]
dist_right_col = new_lc_border_col.astype(np.int16) - final_lc_right[1]
dist_right = np.abs(dist_right_row) + np.abs(dist_right_col)
right_idx = (np.where(dist_right == np.min(dist_right)))
right_idx = right_idx[0][-1]

#print('left idx : {} right idx : {} temp : {}'.format( left_idx, right_idx, np.shape(new_lc_border_row)[0]))

delete_left_lists = np.linspace(0, left_idx,int(left_idx+1), dtype=np.int16)
delete_right_lists = np.linspace(right_idx, np.shape(new_lc_border_row)[0],int(np.shape(new_lc_border_row)[0]- right_idx+1), dtype=np.int16)
concat_delete = np.concatenate((delete_left_lists, delete_right_lists), axis = 0)
print('concat delete : ', concat_delete)
print('delete lists : {}, right : {} shape : {}'.format( delete_left_lists, delete_right_lists, np.shape(delete_right_lists)))
print(np.shape(new_lc_info))
new_final_lc_row = np.delete(new_lc_border_row,  concat_delete)
new_final_lc_col = np.delete(new_lc_border_col,  concat_delete)
print(new_final_lc_col)


copy_result_img[idx, new_final_lc_row, new_final_lc_col] = (125,125,0)
print(new_final_lc_col)

# new
show_on_jupyter(copy_result_img[idx], fig_size=(15,15))

cv2.imwrite(save_folder+filename+'_post_seg.png',copy_result_img[idx])


#new_temp_right = np.delete(new_lc_info, delete_right_lists)

############################################################
############################################################
############################################################
#ym_left_l2_diff[idx] = np.sqrt(np.square((left_gt[0] - left_pred[0]) * _y_convert) + np.square((left_gt[1] - left_pred[1]) * _x_convert) )

no_post_lc_width_arr[idx] = np.linalg.norm(max_end_coord -min_end_coord)
#lc_width_arr[idx] = np.linalg.norm(final_lc_right -final_lc_left)
lc_width_arr[idx] = physical_distance_converter((final_lc_right[0], final_lc_right[1]), (final_lc_left[0], final_lc_left[1]))
print('lc width : {}'.format(lc_width_arr[idx]))
# convert to physical distance
#no_post_lc_width_arr[idx] = np.sqrt(np.square((max_end_coord[0] - min_end_coord[0]) * _y_convert) + np.square((max_end_coord[1] - min_end_coord[1]) * _x_convert) )
#lc_width_arr[idx] = np.linalg.norm(final_lc_right -final_lc_left)

cv2.line(copy_result_img[idx], (final_lc_left[1], final_lc_left[0]), (final_lc_right[1],final_lc_right[0]), (255,0,255), 2 )
cv2.imwrite(save_folder+str(idx)+'_lc_width.png',copy_result_img[idx])
#cv2.line(copy_result_img[idx], (min_end_coord[1], min_end_coord[0]), (max_end_coord[1],max_end_coord[0]), (255,0,255), 2 )

# get laminar cribrosa curve depth final_lc_left, final_lc_right 
print('min end coord : {} max end coord : {} max - min : {} '.format(min_end_coord, max_end_coord, (max_end_coord - min_end_coord)))

max_lc_d , max_lc_coord = get_perpendicular_dist(final_lc_left, final_lc_right, new_final_lc_row,new_final_lc_col)
lc_d_draw_coord = get_intersection_perpendicular_n_line(final_lc_left, final_lc_right, max_lc_coord)
no_post_max_lc_d , no_post_max_lc_coord = get_perpendicular_dist(min_end_coord, max_end_coord, lc_border_row, lc_border_col)

'''
circle_coord = (max_lc_coord[1], max_lc_coord[0])
print(circle_coord)
cv2.circle(copy_result_img[idx] ,circle_coord, 3,(0,255,255),-1)

temp_circle_coord = (lc_d_draw_coord [1], lc_d_draw_coord [0])
cv2.circle(copy_result_img[idx] ,temp_circle_coord, 3,(0,255,255),-1)
'''
cv2.line(copy_result_img[idx], (max_lc_coord[1], max_lc_coord[0]), (lc_d_draw_coord[1], lc_d_draw_coord [0]), (0,255,255), 2 )
cv2.imwrite(save_folder+filename+'_LCCD.png',copy_result_img[idx])

physical_max_lc_d = physical_distance_converter((max_lc_coord[0], max_lc_coord[1]), (lc_d_draw_coord[0], lc_d_draw_coord [1]))
print('physical max d : ', np.shape(physical_max_lc_d))
print('physical max d : ', physical_max_lc_d)
physical_max_lc_d = physical_max_lc_d

left_bmo_coord =np.array((recon_bmo01_center[idx][0] ,recon_bmo01_center[idx][1]))
right_bmo_coord =np.array((recon_bmo02_center[idx][0] ,recon_bmo02_center[idx][1]))

# get laminar cribrosa depth
lc_max_dep_row, lc_max_dep_col = detectPeakPoint(new_final_lc_row, new_final_lc_col)
no_post_lc_max_dep_row, no_post_lc_max_dep_col = detectPeakPoint(lc_border_row, lc_border_col)

max_bmo_d  = getBmoPerpendicularDist(left_bmo_coord,right_bmo_coord,lc_max_dep_row, lc_max_dep_col)
max_bmo_coord = (lc_max_dep_row, lc_max_dep_col)
bmo_d_draw_coord = get_intersection_perpendicular_n_line(left_bmo_coord, right_bmo_coord, max_bmo_coord )


no_post_max_bmo_d  = getBmoPerpendicularDist(left_bmo_coord,right_bmo_coord,no_post_lc_max_dep_row, no_post_lc_max_dep_col)
no_post_max_bmo_coord = (no_post_lc_max_dep_row, no_post_lc_max_dep_col)

#circle_coord02 = (max_bmo_coord[1], max_bmo_coord[0])
#cv2.circle(copy_result_img[idx] ,circle_coord02, 3,(255,255,255),-1)


cv2.line(copy_result_img[idx], (max_bmo_coord[1], max_bmo_coord[0]), (bmo_d_draw_coord[1],bmo_d_draw_coord[0]), (255,255,255), 2 )
print('bmo0 {} bmo1 {} bmoD0 {} bmoD1 {}'.format(max_bmo_coord[0], max_bmo_coord[1],bmo_d_draw_coord[0],bmo_d_draw_coord[1]))

physical_max_bmo_d = physical_distance_converter(np.array((max_bmo_coord[0], max_bmo_coord[1])),  np.array((bmo_d_draw_coord[0],bmo_d_draw_coord[1])))

show_on_jupyter(copy_result_img[idx], fig_size=(15,15))
cv2.imwrite(save_folder+filename+'_final.png', copy_result_img[idx])


print('physical distance LCD : {} LCCD : {} '.format(physical_max_bmo_d, physical_max_lc_d))

print('LCD : {} LCCD : {} LCCI : {} '.format( physical_max_bmo_d,physical_max_lc_d, ((physical_max_lc_d / lc_width_arr[idx])* 100)))
PRED_LCD.append(physical_max_bmo_d)
PRED_LCCD.append(physical_max_lc_d)
PRED_LCCI.append(((physical_max_lc_d / lc_width_arr[idx])* 100))

no_post_PRED_LCD.append(no_post_max_bmo_d)
no_post_PRED_LCCD.append(no_post_max_lc_d)
no_post_PRED_LCCI.append(((no_post_max_lc_d / no_post_lc_width_arr[idx])* 100))
    

f = open(save_folder+"result.txt", 'w')
f.write('LCD : ' + str(physical_max_bmo_d) + '\n' +'LCCD : '+str(physical_max_lc_d[0]) + '\n' + 'LCCI : ' + str(((physical_max_lc_d[0] / lc_width_arr[idx])* 100)))
f.close()

In [ ]:
if VALID_TEST == 'test':
    LC_NAME = 'N20'
    lc_save_folder = './case_study/'+LC_NAME+'/'
    dir_checker(lc_save_folder)
    
    
_x_convert = 12.5000
_y_convert = 3.9216

lc_width_arr = np.zeros(NUM_YOLO_RESULT, dtype=np.float32)
no_post_lc_width_arr = np.zeros(NUM_YOLO_RESULT, dtype=np.float32)


copy_result_img = RESULT_IMGS.copy()
copy_result_mask = RESULT_IMGS_MASKS.copy()

PRED_LCD = []
PRED_LCCD = []
PRED_LCCI = []

no_post_PRED_LCD = []
no_post_PRED_LCCD = []
no_post_PRED_LCCI = []

dash_constant = 5

# result image precessing
#for idx in range(NUM_YOLO_RESULT):
for idx in range(NUM_YOLO_RESULT):
    idx = 6
    temp_left = recon_bmo01_center[idx]
    temp_right = recon_bmo02_center[idx]
    
    # change the left & right
    if recon_bmo01_center[idx][1] > recon_bmo02_center[idx][1]:
        recon_bmo01_center[idx] = temp_right
        recon_bmo02_center[idx] = temp_left
    print('recon bmo center type : ', type(recon_bmo01_center))
    
    cv2.imwrite(save_folder+str(idx)+'_lc_img.png',copy_result_img[idx])
    # debugging process
    RESULT_IMGS[idx, recon_bmo01_center[idx][0], recon_bmo01_center[idx][1]] = (0,0,255)
    RESULT_IMGS[idx, recon_bmo02_center[idx][0], recon_bmo02_center[idx][1]] = (0,0,255) 
    copy_result_img[idx, recon_bmo01_center[idx][0], recon_bmo01_center[idx][1]] = (0,0,255)
    copy_result_img[idx, recon_bmo02_center[idx][0], recon_bmo02_center[idx][1]] = (0,0,255) 
    
    
    L_bmo_circle_coord = (recon_bmo01_center[idx][1], recon_bmo01_center[idx][0])
    R_bmo_circle_coord = (recon_bmo02_center[idx][1], recon_bmo02_center[idx][0])
    
    cv2.circle(copy_result_img[idx] ,L_bmo_circle_coord, 3,(0,0,255),-1)
    cv2.circle(copy_result_img[idx] ,R_bmo_circle_coord, 3,(0,0,255),-1)
    show_on_jupyter(copy_result_img[idx], fig_size= (17,17))
    cv2.imwrite(save_folder+str(idx)+'_seg.png',copy_result_img[idx])
    if VALID_TEST == 'test':
        cv2.imwrite(lc_save_folder+LC_NAME+'_'+str(idx)+'_seg.png', copy_result_img[idx])
        
    
    cv2.line(copy_result_img[idx], (recon_bmo01_center[idx][1], recon_bmo01_center[idx][0]), (recon_bmo02_center[idx][1], recon_bmo02_center[idx][0]), (255,255,0), 2 )
    cv2.imwrite(save_folder+str(idx)+'_bmo_ref.png',copy_result_img[idx])
    ################################################################################################
    # perpendicular line to bmo terminate points
    line_coord = createLineIterator(np.asarray(recon_bmo01_center[idx]), np.asarray(recon_bmo02_center[idx]), copy_result_img[idx,:,:,0])

    slope = getSlopeVal(np.asarray(recon_bmo01_center[idx]), np.asarray(recon_bmo02_center[idx]))
    print('slope : ' , slope)
    left_target_point,left_b = getPerpendicularLineList(np.asarray(recon_bmo01_center[idx]), slope,target_y=450)
    left_per_line_coord, max_left_line_points = createLineIterator(np.asarray(recon_bmo01_center[idx]), left_target_point, copy_result_img[idx,:,:,0], flag = 'min')
    left_dash_line_coord = getDashLine(left_per_line_coord, dash_constant=dash_constant, delete_connect_val=3)
    copy_result_img[idx,left_dash_line_coord[:,0].astype(np.uint16), left_dash_line_coord[:,1].astype(np.uint16)] = (255,255,255)
    #cv2.circle(copy_result_img[idx] ,(recon_bmo01_center[idx][1], recon_bmo01_center[idx][0]), 5,(0,255,255))
    
    right_target_point,right_b = getPerpendicularLineList(np.asarray(recon_bmo02_center[idx]), slope,target_y=450)
    right_per_line_coord, max_right_line_points = createLineIterator(np.asarray(recon_bmo02_center[idx]), right_target_point, copy_result_img[idx,:,:,0], flag ='max')
    right_dash_line_coord = getDashLine(right_per_line_coord, dash_constant=dash_constant, delete_connect_val=3)
    copy_result_img[idx,right_dash_line_coord[:,0].astype(np.uint16), right_dash_line_coord[:,1].astype(np.uint16)] = (255,255,255)
    
    ################################################################################################
    
    RESULT_IMGS_MASKS[idx, recon_bmo01_center[idx][0], recon_bmo01_center[idx][1]] = (0,0,255)
    RESULT_IMGS_MASKS[idx, recon_bmo02_center[idx][0], recon_bmo02_center[idx][1]] = (0,0,255)
    
    # LC
    lc_border_row, lc_border_col = np.where(np.all(RESULT_IMGS[idx] == (0,255,0), axis=-1))
    ########################## 잠깐 실험...! (polyfit)
    print('left line info : {} right line info :{}'.format(max_left_line_points, max_right_line_points))
    
    left_interpol_point = 0
    right_interpol_point = 0
    if int(max_left_line_points[1]) >  recon_bmo01_center[idx][1]:
        left_interpol_point = recon_bmo01_center[idx][1]
    else:
        print('exception! : {}'.format(max_left_line_points[1]))
        left_interpol_point = int(max_left_line_points[1])
        
        
    if int(max_right_line_points[1]) < recon_bmo02_center[idx][1]:
        right_interpol_point = recon_bmo02_center[idx][1]
    else:
        right_interpol_point = int(max_right_line_points[1])
        
    print('right interpol point : ', right_interpol_point)
    show_on_jupyter(copy_result_img[idx], fig_size=(17,17))
    cv2.imwrite(save_folder+str(idx)+'_lc_interpol.png',copy_result_img[idx])
    ################################################################################################
    
    L_per_info = (left_per_line_coord[:,0].astype(np.uint16), left_per_line_coord[:,1].astype(np.uint16))
    R_per_info = (right_per_line_coord[:,0].astype(np.uint16), right_per_line_coord[:,1].astype(np.uint16))
    left, right = getWidthPoint(np.shape(YOLO_IMGS),(lc_border_row,lc_border_col),L_per_info, R_per_info)
    print('left : {} right : {}'.format(left, right))
    # 흔적
    
    min_end_coord = left
    max_end_coord = right
    print('min end coord : {} max end coord {} '.format(min_end_coord,max_end_coord))
    ##################### perpendicular ####################
    ############################################################
    ############################################################
    
    left_end_lc_point = left
    right_end_lc_point = right
    print('right end lc point[1] : {} right interpol point : {}'.format(right_end_lc_point[1], right_interpol_point))
    interpol_lc_left_x_range = np.linspace(left_interpol_point, left_end_lc_point[1],int(left_end_lc_point[1] -left_interpol_point+1))
    interpol_lc_right_x_range = np.linspace(right_end_lc_point[1], right_interpol_point,int(right_interpol_point-right_end_lc_point[1]+1))
    print('idx : ',idx)
    print('left interpolation range : {} \n right interpolation range : {}'.format(interpol_lc_left_x_range, interpol_lc_right_x_range))
    
    #print('reshape : {} pred coef[idx] : {} '.format(interpol_lc_left_x_range.reshape(-1,1), pred_coef_lists[idx]))
    new_pred_left_lc_y = pred_coef_lists[idx].predict(interpol_lc_left_x_range.reshape(-1,1))
    new_pred_left_lc_y = np.around(new_pred_left_lc_y[:,0]).astype(np.uint16)
    
    new_pred_right_lc_y = pred_coef_lists[idx].predict(interpol_lc_right_x_range.reshape(-1,1))
    new_pred_right_lc_y = np.around(new_pred_right_lc_y[:,0]).astype(np.uint16)
    
    print('pred cocl shape : {} new pred left lc x shape : {}'.format(lc_border_col.shape, new_pred_left_lc_y.shape))
    
    new_lc_border_col = np.concatenate((interpol_lc_left_x_range, lc_border_col),axis = 0)
    new_lc_border_col = np.concatenate((new_lc_border_col, interpol_lc_right_x_range), axis =0)
    new_lc_border_col = new_lc_border_col.astype(np.uint16)
    
    new_lc_border_row = np.concatenate((new_pred_left_lc_y, lc_border_row), axis =0 )
    new_lc_border_row = np.concatenate((new_lc_border_row, new_pred_right_lc_y), axis =0 )

    print('concat row shape : {} concat col shape : {}'.format(new_lc_border_row.shape, new_lc_border_col.shape))
    
    #new_lc_border_row[new_lc_border_row > 500]
    #copy_result_img[idx, new_lc_border_row, new_lc_border_col] = (125,125,0)
    #show_on_jupyter(copy_result_img[idx], fig_size=(15,15))
    print('concat row : {} concat col: {}'.format(new_lc_border_row, new_lc_border_col))
    final_lc_left, final_lc_right = getWidthPoint(np.shape(YOLO_IMGS),(new_lc_border_row,new_lc_border_col),L_per_info, R_per_info)

    # find final lc point's index
    print('final lc left : {}'.format(final_lc_left))
    print('row : {} \n\n col : {}'.format(new_lc_border_row, new_lc_border_col))
    # delete, 찾은 값 기준 범위
    left_col_idx = np.where( new_lc_border_col == final_lc_left[1])
    left_row_idx = np.where( new_lc_border_row == final_lc_left[0])
    print('final lc left col idx : {} row idx : {}'.format(left_col_idx, left_row_idx))
    
    new_lc_info = np.array((new_lc_border_row, new_lc_border_col))
    dist_left_row = new_lc_border_row - final_lc_left[0]
    dist_left_col = new_lc_border_col.astype(np.int16) - final_lc_left[1]
    dist_left = np.abs(dist_left_row) + np.abs(dist_left_col)
    left_idx = (np.where(dist_left == np.min(dist_left)))
    left_idx = left_idx[0][0]
    
    dist_right_row = new_lc_border_row - final_lc_right[0]
    dist_right_col = new_lc_border_col.astype(np.int16) - final_lc_right[1]
    dist_right = np.abs(dist_right_row) + np.abs(dist_right_col)
    right_idx = (np.where(dist_right == np.min(dist_right)))
    right_idx = right_idx[0][-1]
    
    #print('left idx : {} right idx : {} temp : {}'.format( left_idx, right_idx, np.shape(new_lc_border_row)[0]))
    
    delete_left_lists = np.linspace(0, left_idx,int(left_idx+1), dtype=np.int16)
    delete_right_lists = np.linspace(right_idx, np.shape(new_lc_border_row)[0],int(np.shape(new_lc_border_row)[0]- right_idx+1), dtype=np.int16)
    concat_delete = np.concatenate((delete_left_lists, delete_right_lists), axis = 0)
    print('concat delete : ', concat_delete)
    print('delete lists : {}, right : {} shape : {}'.format( delete_left_lists, delete_right_lists, np.shape(delete_right_lists)))
    print(np.shape(new_lc_info))
    new_final_lc_row = np.delete(new_lc_border_row,  concat_delete)
    new_final_lc_col = np.delete(new_lc_border_col,  concat_delete)
    print(new_final_lc_col)
    
    
    copy_result_img[idx, new_final_lc_row, new_final_lc_col] = (125,125,0)
    print(new_final_lc_col)
    
    # new
    show_on_jupyter(copy_result_img[idx], fig_size=(15,15))
    
    cv2.imwrite(save_folder+str(idx)+'_post_seg.png',copy_result_img[idx])
    if VALID_TEST == 'test':
        cv2.imwrite(lc_save_folder+LC_NAME+'_'+str(idx)+'_post_seg.png', copy_result_img[idx])
    
    #new_temp_right = np.delete(new_lc_info, delete_right_lists)
    
    ############################################################
    ############################################################
    ############################################################
    #ym_left_l2_diff[idx] = np.sqrt(np.square((left_gt[0] - left_pred[0]) * _y_convert) + np.square((left_gt[1] - left_pred[1]) * _x_convert) )

    no_post_lc_width_arr[idx] = np.linalg.norm(max_end_coord -min_end_coord)
    #lc_width_arr[idx] = np.linalg.norm(final_lc_right -final_lc_left)
    lc_width_arr[idx] = physical_distance_converter((final_lc_right[0], final_lc_right[1]), (final_lc_left[0], final_lc_left[1]))
    print('lc width : {}'.format(lc_width_arr[idx]))
    # convert to physical distance
    #no_post_lc_width_arr[idx] = np.sqrt(np.square((max_end_coord[0] - min_end_coord[0]) * _y_convert) + np.square((max_end_coord[1] - min_end_coord[1]) * _x_convert) )
    #lc_width_arr[idx] = np.linalg.norm(final_lc_right -final_lc_left)
    
    cv2.line(copy_result_img[idx], (final_lc_left[1], final_lc_left[0]), (final_lc_right[1],final_lc_right[0]), (255,0,255), 2 )
    cv2.imwrite(save_folder+str(idx)+'_lc_width.png',copy_result_img[idx])
    #cv2.line(copy_result_img[idx], (min_end_coord[1], min_end_coord[0]), (max_end_coord[1],max_end_coord[0]), (255,0,255), 2 )
    
    # get laminar cribrosa curve depth final_lc_left, final_lc_right 
    print('min end coord : {} max end coord : {} max - min : {} '.format(min_end_coord, max_end_coord, (max_end_coord - min_end_coord)))
    
    max_lc_d , max_lc_coord = get_perpendicular_dist(final_lc_left, final_lc_right, new_final_lc_row,new_final_lc_col)
    lc_d_draw_coord = get_intersection_perpendicular_n_line(final_lc_left, final_lc_right, max_lc_coord)
    no_post_max_lc_d , no_post_max_lc_coord = get_perpendicular_dist(min_end_coord, max_end_coord, lc_border_row, lc_border_col)
    
    '''
    circle_coord = (max_lc_coord[1], max_lc_coord[0])
    print(circle_coord)
    cv2.circle(copy_result_img[idx] ,circle_coord, 3,(0,255,255),-1)
    
    temp_circle_coord = (lc_d_draw_coord [1], lc_d_draw_coord [0])
    cv2.circle(copy_result_img[idx] ,temp_circle_coord, 3,(0,255,255),-1)
    '''
    cv2.line(copy_result_img[idx], (max_lc_coord[1], max_lc_coord[0]), (lc_d_draw_coord[1], lc_d_draw_coord [0]), (0,255,255), 2 )
    cv2.imwrite(save_folder+str(idx)+'_LCCD.png',copy_result_img[idx])
    physical_max_lc_d = physical_distance_converter((max_lc_coord[0], max_lc_coord[1]), (lc_d_draw_coord[0], lc_d_draw_coord [1]))
    print('physical max d : ', np.shape(physical_max_lc_d))
    print('physical max d : ', physical_max_lc_d)
    physical_max_lc_d = physical_max_lc_d

    left_bmo_coord =np.array((recon_bmo01_center[idx][0] ,recon_bmo01_center[idx][1]))
    right_bmo_coord =np.array((recon_bmo02_center[idx][0] ,recon_bmo02_center[idx][1]))
    
    # get laminar cribrosa depth
    lc_max_dep_row, lc_max_dep_col = detectPeakPoint(new_final_lc_row, new_final_lc_col)
    no_post_lc_max_dep_row, no_post_lc_max_dep_col = detectPeakPoint(lc_border_row, lc_border_col)
    
    max_bmo_d  = getBmoPerpendicularDist(left_bmo_coord,right_bmo_coord,lc_max_dep_row, lc_max_dep_col)
    max_bmo_coord = (lc_max_dep_row, lc_max_dep_col)
    bmo_d_draw_coord = get_intersection_perpendicular_n_line(left_bmo_coord, right_bmo_coord, max_bmo_coord )

    
    no_post_max_bmo_d  = getBmoPerpendicularDist(left_bmo_coord,right_bmo_coord,no_post_lc_max_dep_row, no_post_lc_max_dep_col)
    no_post_max_bmo_coord = (no_post_lc_max_dep_row, no_post_lc_max_dep_col)
    
    #circle_coord02 = (max_bmo_coord[1], max_bmo_coord[0])
    #cv2.circle(copy_result_img[idx] ,circle_coord02, 3,(255,255,255),-1)
    
    
    cv2.line(copy_result_img[idx], (max_bmo_coord[1], max_bmo_coord[0]), (bmo_d_draw_coord[1],bmo_d_draw_coord[0]), (255,255,255), 2 )
    print('bmo0 {} bmo1 {} bmoD0 {} bmoD1 {}'.format(max_bmo_coord[0], max_bmo_coord[1],bmo_d_draw_coord[0],bmo_d_draw_coord[1]))

    physical_max_bmo_d = physical_distance_converter(np.array((max_bmo_coord[0], max_bmo_coord[1])),  np.array((bmo_d_draw_coord[0],bmo_d_draw_coord[1])))
    
    show_on_jupyter(copy_result_img[idx], fig_size=(15,15))
    cv2.imwrite(save_folder+str(idx)+'_final.png', copy_result_img[idx])
    if VALID_TEST == 'test':
        cv2.imwrite(lc_save_folder+LC_NAME+'_'+str(idx)+'_final.png', copy_result_img[idx])
        
    print('physical distance LCD : {} LCCD : {} '.format(physical_max_bmo_d, physical_max_lc_d))
    
    print('LCD : {} LCCD : {} LCCI : {} '.format( physical_max_bmo_d,physical_max_lc_d, ((physical_max_lc_d / lc_width_arr[idx])* 100)))
    PRED_LCD.append(physical_max_bmo_d)
    PRED_LCCD.append(physical_max_lc_d)
    PRED_LCCI.append(((physical_max_lc_d / lc_width_arr[idx])* 100))
    
    no_post_PRED_LCD.append(no_post_max_bmo_d)
    no_post_PRED_LCCD.append(no_post_max_lc_d)
    no_post_PRED_LCCI.append(((no_post_max_lc_d / no_post_lc_width_arr[idx])* 100))
    
    

    

In [ ]:
LC_PARAM_TABLE = pd.DataFrame(columns= ['LCD', 'LCCD', 'LCCI'])
AVG_LC_PARAM_TABLE = pd.DataFrame(columns= ['AVG_LCD', 'AVG_LCCD', 'AVG_LCCI'])


#8,7,6,5,4


for idx in range(NUM_YOLO_RESULT):
    LC_PARAM_TABLE.loc[idx,"LCD"] = PRED_LCD[idx]
    LC_PARAM_TABLE.loc[idx,"LCCD"] = PRED_LCCD[idx]
    LC_PARAM_TABLE.loc[idx,"LCCI"] = PRED_LCCI[idx]

fig,ax = render_mpl_table(LC_PARAM_TABLE, header_columns=0, col_width=5.0)
fig.savefig(lc_save_folder +LC_NAME+ '_LC_PARAM_TABLE.png')

AVG_LC_PARAM_TABLE.loc[idx,"AVG_LCD"] = np.average(PRED_LCD)
AVG_LC_PARAM_TABLE.loc[idx,"AVG_LCCD"] = np.average(PRED_LCCD)
AVG_LC_PARAM_TABLE.loc[idx,"AVG_LCCI"] = np.average(PRED_LCCI)
fig,ax = render_mpl_table(AVG_LC_PARAM_TABLE, header_columns=0, col_width=5.0)
fig.savefig(lc_save_folder +LC_NAME+ '_AVG_LC_PARAM_TABLE.png')

### Clinical parameters
- 최종적인 목표는 ground <-> pred blend-altman plot
- 위에서 pred는 (pixel-wise) 구한 상태
- gt를 구해야함


In [ ]:
def peakdetect(y_axis, x_axis = None, lookahead = 500, delta = 0):
    """
    Converted from/based on a MATLAB script at http://billauer.co.il/peakdet.html
    
    Algorithm for detecting local maximas and minmias in a signal.
    Discovers peaks by searching for values which are surrounded by lower
    or larger values for maximas and minimas respectively
    
    keyword arguments:
    y_axis -- A list containg the signal over which to find peaks
    x_axis -- A x-axis whose values correspond to the 'y_axis' list and is used
        in the return to specify the postion of the peaks. If omitted the index
        of the y_axis is used. (default: None)
    lookahead -- (optional) distance to look ahead from a peak candidate to
        determine if it is the actual peak (default: 500) 
        '(sample / period) / f' where '4 >= f >= 1.25' might be a good value
    delta -- (optional) this specifies a minimum difference between a peak and
        the following points, before a peak may be considered a peak. Useful
        to hinder the algorithm from picking up false peaks towards to end of
        the signal. To work well delta should be set to 'delta >= RMSnoise * 5'.
        (default: 0)
            Delta function causes a 20% decrease in speed, when omitted
            Correctly used it can double the speed of the algorithm
    
    return -- two lists [maxtab, mintab] containing the positive and negative
        peaks respectively. Each cell of the lists contains a tupple of:
        (position, peak_value) 
        to get the average peak value do 'np.mean(maxtab, 0)[1]' on the results
    """
    maxtab = []
    mintab = []
    dump = []   #Used to pop the first hit which always if false
       
    length = len(y_axis)
    if x_axis is None:
        x_axis = range(length)
    
    #perform some checks
    if length != len(x_axis):
        raise ValueError("Input vectors y_axis and x_axis must have same length")
    if lookahead < 1:
        raise ValueError("Lookahead must be above '1' in value")
    if not (np.isscalar(delta) and delta >= 0):
        raise ValueError("delta must be a positive number")
    
    #needs to be a numpy array
    y_axis = np.asarray(y_axis)
    
    #maxima and minima candidates are temporarily stored in
    #mx and mn respectively
    mn, mx = np.Inf, -np.Inf
    
    #Only detect peak if there is 'lookahead' amount of points after it
    for index, (x, y) in enumerate(zip(x_axis[:-lookahead], y_axis[:-lookahead])):
        if y > mx:
            mx = y
            mxpos = x
        if y < mn:
            mn = y
            mnpos = x
        
        ####look for max####
        if y < mx-delta and mx != np.Inf:
            #Maxima peak candidate found
            #look ahead in signal to ensure that this is a peak and not jitter
            if y_axis[index:index+lookahead].max() < mx:
                maxtab.append((mxpos, mx))
                dump.append(True)
                #set algorithm to only find minima now
                mx = np.Inf
                mn = np.Inf
        
        ####look for min####
        if y > mn+delta and mn != -np.Inf:
            #Minima peak candidate found 
            #look ahead in signal to ensure that this is a peak and not jitter
            if y_axis[index:index+lookahead].min() > mn:
                mintab.append((mnpos, mn))
                dump.append(False)
                #set algorithm to only find maxima now
                mn = -np.Inf
                mx = -np.Inf
    
    
    #Remove the false hit on the first value of the y_axis
    try:
        if dump[0]:
            maxtab.pop(0)
            #print "pop max"
        else:
            mintab.pop(0)
            #print "pop min"
        del dump
    except IndexError:
        #no peaks were found, should the function return empty lists?
        pass
    
    return maxtab, mintab

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2))

In [ ]:
from skimage.morphology import medial_axis, skeletonize
from scipy.signal import find_peaks
from scipy.signal import find_peaks_cwt

gt_lc_width_arr = np.zeros(NUM_YOLO_RESULT, dtype=np.float32)

copy_gt_img = GROUND_TRUTH_IMG.copy()
copy_ori_img = real_copy_ori_img.copy()

GT_LCD = []
GT_LCCD = []
GT_LCCI = []

avg_rmse_lists = []
mean_peak = []

for idx in range(NUM_YOLO_RESULT):
    
    left_bmo_center = np.asarray((LEFT_GROUND_CENTER[idx][0], LEFT_GROUND_CENTER[idx][1]))
    right_bmo_center = np.asarray((RIGHT_GROUND_CENTER[idx][0], RIGHT_GROUND_CENTER[idx][1]))
    print('bmo center type : {} shape : {}'.format(type(left_bmo_center), np.shape(left_bmo_center)))
    
    
    copy_ori_img[idx,left_bmo_center[0], left_bmo_center[1]] = (0,0,255)
    copy_ori_img[idx, right_bmo_center[0], right_bmo_center[1]] = (0,0,255) 
    show_on_jupyter(copy_gt_img[idx])
    
    
    cv2.line(copy_ori_img[idx], (left_bmo_center[1], left_bmo_center[0]), (right_bmo_center[1], right_bmo_center[0]), (255,255,0), 2 )

    # LC
    lc_border_row, lc_border_col = np.where(np.all(GROUND_TRUTH_IMG[idx] == (255,0,0), axis=-1))
    gray_mask = np.zeros((SHAPE_YOLO_RESULT[0], SHAPE_YOLO_RESULT[1]))
    gray_mask[lc_border_row, lc_border_col] = True
    gray_mask = skeletonize(gray_mask)
    skeleton_row, skeleton_col = np.nonzero(gray_mask)
    copy_ori_img[idx, skeleton_row, skeleton_col, :] = (0,255,0)
    
    #show_on_jupyter(gray_mask*255,'gray')

    ################################################################################################
    # perpendicular line to bmo terminate points
    line_coord = createLineIterator(np.asarray(left_bmo_center), np.asarray(right_bmo_center), copy_ori_img[idx,:,:,0])

    dash_constant = 5
    slope = getSlopeVal(np.asarray(left_bmo_center), np.asarray(right_bmo_center))
    left_target_point,left_b = getPerpendicularLineList(np.asarray(left_bmo_center), slope,target_y=450)
    
    left_per_line_coord = createLineIterator(np.asarray(left_bmo_center), left_target_point, copy_ori_img[idx,:,:,0])
    left_dash_line_coord = getDashLine(left_per_line_coord, dash_constant=dash_constant, delete_connect_val=3)
    copy_ori_img[idx,left_dash_line_coord[:,0].astype(np.uint16), left_dash_line_coord[:,1].astype(np.uint16)] = (255,255,255)
    #cv2.circle(copy_result_img[idx] ,(recon_bmo01_center[idx][1], recon_bmo01_center[idx][0]), 5,(0,255,255))

    right_target_point,right_b = getPerpendicularLineList(np.asarray(right_bmo_center), slope,target_y=450)
    right_per_line_coord = createLineIterator(np.asarray(right_bmo_center), right_target_point, copy_ori_img[idx,:,:,0])
    right_dash_line_coord = getDashLine(right_per_line_coord, dash_constant=dash_constant, delete_connect_val=3)
    copy_ori_img[idx,right_dash_line_coord[:,0].astype(np.uint16), right_dash_line_coord[:,1].astype(np.uint16)] = (255,255,255)
    
    ################################################################################################
    
    

    #min_col_idx = np.where(skeleton_col==np.min(skeleton_col))
    #max_col_idx = np.where(skeleton_col == np.max(skeleton_col))
    
    lc_border_row, lc_border_col = np.where(np.all(copy_ori_img[idx] == (0,255,0), axis=-1))
    show_on_jupyter(copy_ori_img[idx])
    
    L_per_info = (left_per_line_coord[:,0].astype(np.uint16), left_per_line_coord[:,1].astype(np.uint16))
    R_per_info = (right_per_line_coord[:,0].astype(np.uint16), right_per_line_coord[:,1].astype(np.uint16))
    left, right = getWidthPoint(np.shape(YOLO_IMGS),(lc_border_row,lc_border_col),L_per_info, R_per_info)
    
    
    #min_end_coord =np.array((skeleton_row[min_col_idx[0][0]] ,skeleton_col[min_col_idx[0][0]]))
    #max_end_coord =np.array((skeleton_row[max_col_idx[0][0]] ,skeleton_col[max_col_idx[0][0]]))
    min_end_coord = left
    max_end_coord = right
    gt_lc_width_arr[idx] = np.linalg.norm(max_end_coord - min_end_coord)
    
    
    cv2.line(copy_ori_img[idx], (min_end_coord[1], min_end_coord[0]), (max_end_coord[1],max_end_coord[0]), (255,0,255), 2 )

    max_lc_d , max_lc_coord = get_perpendicular_dist(min_end_coord, max_end_coord,skeleton_row, skeleton_col)
    circle_coord = (max_lc_coord[1], max_lc_coord[0])
    print(circle_coord)
    print('circle : ', circle_coord[0], circle_coord[1])
    cv2.circle(copy_ori_img[idx] ,circle_coord, 3,(0,255,255))
    
    
    left_bmo_coord =np.array((left_bmo_center[0] ,left_bmo_center[1]))
    right_bmo_coord =np.array((right_bmo_center[0] ,right_bmo_center[1]))
    
    
    lc_max_dep_row, lc_max_dep_col = detectPeakPoint(skeleton_row, skeleton_col)
    max_bmo_d  = getBmoPerpendicularDist(left_bmo_coord,right_bmo_coord,lc_max_dep_row, lc_max_dep_col)
    max_bmo_coord = (lc_max_dep_row, lc_max_dep_col)
    circle_coord02 = (max_bmo_coord[1], max_bmo_coord[0])
    
    #max_bmo_d , max_bmo_coord = get_perpendicular_dist(left_bmo_coord,right_bmo_coord,lc_border_row, lc_border_col)
    #circle_coord02 = (max_bmo_coord[1], max_bmo_coord[0])
    
    cv2.circle(copy_ori_img[idx] ,circle_coord02, 3,(255,255,255))
    show_on_jupyter(copy_ori_img[idx], fig_size=(13,13))
    
    print('LCD : {} LCCD : {} LCCI : {} '.format( max_bmo_d,max_lc_d, ((max_lc_d / gt_lc_width_arr[idx])* 100)))
    GT_LCD.append(max_bmo_d)
    GT_LCCD.append(max_lc_d)
    GT_LCCI.append(((max_lc_d / gt_lc_width_arr[idx])* 100))
print('aveage rmse : ', np.mean(np.array(avg_rmse_lists)))
print('mean peak : ',np.mean(np.array(mean_peak)))

In [ ]:
np.divide(PRED_LCD,GT_LCD)

In [ ]:
print(np.average(np.divide(PRED_LCD,GT_LCD)))
print(np.average(np.divide(PRED_LCCD,GT_LCCD)))
print(np.average(np.divide(PRED_LCCI,GT_LCCI)))
print(np.average(np.divide(lc_width_arr, gt_lc_width_arr)))


print(np.average(np.divide(no_post_PRED_LCD,GT_LCD)))
print(np.average(np.divide(no_post_PRED_LCCD,GT_LCCD)))
print(np.average(np.divide(no_post_PRED_LCCI,GT_LCCI)))
print(np.average(np.divide(no_post_lc_width_arr, gt_lc_width_arr)))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def bland_altman_plot(data1, data2, *args, **kwargs):
    data1     = np.asarray(data1)
    data2     = np.asarray(data2)
    mean      = np.mean([data1, data2], axis=0)
    diff      = data1 - data2                   # Difference between data1 and data2
    md        = np.mean(diff)                   # Mean of the difference
    sd        = np.std(diff, axis=0)            # Standard deviation of the difference

    plt.scatter(mean, diff, *args, **kwargs)
    plt.axhline(md,           color='gray', linestyle='--')
    plt.axhline(md + 1.96*sd, color='gray', linestyle='--')
    plt.axhline(md - 1.96*sd, color='gray', linestyle='--')

In [ ]:
from numpy.random import random

bland_altman_plot(np.array(GT_LCD),np.array(PRED_LCD))
plt.title('Bland-Altman Plot')
plt.show()


In [ ]:
bland_altman_plot(np.array(GT_LCCD),np.array(PRED_LCCD))
plt.title('Bland-Altman Plot')
plt.show()

In [ ]:
bland_altman_plot(np.array(GT_LCCI),np.array(PRED_LCCI))
plt.title('Bland-Altman Plot')
plt.show()

In [ ]:
import statsmodels.api as sm
save_folder = './pipe_result/'+str(name_experiment)+'/'
print(save_folder)
f, ax = plt.subplots(1, figsize = (8,5))
sm.graphics.mean_diff_plot(np.array(GT_LCD), np.array(PRED_LCD), ax = ax)
plt.savefig(save_folder+str(spline_order) + '_LCD_bland_altman.png')
plt.show()

In [ ]:
f, ax = plt.subplots(1, figsize = (8,5))
sm.graphics.mean_diff_plot(np.array(GT_LCCD), np.array(PRED_LCCD), ax = ax)
plt.savefig(save_folder +str(spline_order)+ 'LCCD_bland_altman.png')
plt.show()

In [ ]:
f, ax = plt.subplots(1, figsize = (8,5))
sm.graphics.mean_diff_plot(np.array(GT_LCCI), np.array(PRED_LCCI), ax = ax)
plt.savefig(save_folder +str(spline_order)+ '_LCCI_bland_altman.png')

plt.show()

In [ ]:
# width blend altman plot

f, ax = plt.subplots(1, figsize = (8,5))
sm.graphics.mean_diff_plot(np.array(gt_lc_width_arr), np.array(lc_width_arr), ax = ax)
plt.savefig(save_folder +str(spline_order)+ '_width_bland_altman.png')
plt.show()

Glaucoma_OCT

In [ ]:
# width blend altman plot

f, ax = plt.subplots(1, figsize = (8,5))
sm.graphics.mean_diff_plot(np.array(gt_lc_width_arr), np.array(no_post_lc_width_arr), ax = ax)
plt.savefig(save_folder +str(spline_order)+ '_no_post_width_bland_altman.png')
plt.show()



In [ ]:
lc_width_arr

In [ ]:
gt_lc_width_arr

In [ ]:
no_post_lc_width_arr

In [ ]:
import scipy

LCCD_PEARSON = scipy.stats.pearsonr(GT_LCCD, PRED_LCCD)
LCD_PEARSON = scipy.stats.pearsonr(GT_LCD, PRED_LCD)
LCCI_PEARSON = scipy.stats.pearsonr(GT_LCCI, PRED_LCCI)

print(LCCD_PEARSON)
print(LCD_PEARSON)
print(LCCI_PEARSON)

print(LCD_PEARSON[0]**2)
print(LCCI_PEARSON[0]**2)

In [ ]:
from scipy import stats

column_A= GT_LCCD
column_B= PRED_LCCD
df = pd.DataFrame({'A': column_A, 'B': column_B})

reg = stats.linregress(df.A, df.B)

plt.plot(df.A, df.B, 'bo', label='Data')
plt.plot(df.A, reg.intercept + reg.slope * df.A, 'k-', label='Linear Regression')
plt.xlabel('Ground-truth')
plt.ylabel('Prediction')
plt.text(0.000001, 0.45,'R-squared : {} \np-value < 0.01'.format(round(LCCD_PEARSON[0]**2,4)), transform=ax.transAxes)
plt.legend()
plt.savefig(save_folder+str(spline_order)+'_LCCD_correlation.png')
plt.show()

In [ ]:
column_A= GT_LCD
column_B= PRED_LCD
df = pd.DataFrame({'A': column_A, 'B': column_B})

reg = stats.linregress(df.A, df.B)

plt.plot(df.A, df.B, 'bo', label='Data')
plt.plot(df.A, reg.intercept + reg.slope * df.A, 'k-', label='Linear Regression')
plt.xlabel('Ground-truth')
plt.ylabel('Prediction')
plt.text(0.000001, 0.45,'R-squared : {} \np-value < 0.01'.format(round(LCD_PEARSON[0]**2,4)), transform=ax.transAxes)
plt.legend()
plt.savefig(save_folder+str(spline_order)+'_LCD_correlation.png')
plt.show()

In [ ]:
column_A= GT_LCCI
column_B= PRED_LCCI
df = pd.DataFrame({'A': column_A, 'B': column_B})

reg = stats.linregress(df.A, df.B)

plt.plot(df.A, df.B, 'bo', label='Data')
plt.plot(df.A, reg.intercept + reg.slope * df.A, 'k-', label='Linear Regression')
plt.xlabel('Ground-truth')
plt.ylabel('Prediction')
plt.text(0.000001, 0.45,'R-squared : {} \np-value < 0.01'.format(round(LCCI_PEARSON[0]**2,4)), transform=ax.transAxes)
plt.legend()
plt.savefig(save_folder+str(spline_order)+'_LCCI_correlation.png')
plt.show()

In [ ]:
clinical_table = pd.DataFrame(columns=['Class','Correlation factor (R)','P-value'])
clinical_table.loc[0,'Class' ] = 'LCCD'
clinical_table.loc[0,'Correlation factor (R)' ] = LCCD_PEARSON[0]
clinical_table.loc[0,'P-value' ] = LCCD_PEARSON[1]

clinical_table.loc[1,'Class' ] = 'LCD'
clinical_table.loc[1,'Correlation factor (R)' ] = LCD_PEARSON[0]
clinical_table.loc[1,'P-value' ] = LCD_PEARSON[1]

clinical_table.loc[2,'Class' ] = 'LCCI'
clinical_table.loc[2,'Correlation factor (R)' ] = LCCI_PEARSON[0]
clinical_table.loc[2,'P-value' ] = LCCI_PEARSON[1]

clinical_table.to_excel(save_folder +str(spline_order)+ "_clinical_table.xlsx")  
clinical_table.to_csv(save_folder +str(spline_order)+ "_clinical_table.csv") 

In [ ]:
clinical_table

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = [4,5,6,7,8]
normal_lcd = [405.72, 411.32, 376.99, 392.13, 403.07]
glaucoma_lcd = [635.73,566.47,598.38,674.24,672.96]

normal_lccd = [23.53,43.14,52.49,33.77,47.06]
glaucoma_lccd = [178.44,111.39,124.23,127.83,194.51]

normal_lcci = [1.39,4.48,3.13,2.11,3.49]
glaucoma_lcci = [7.74,4.90,7.41,5.40,9.12]

plt.figure(figsize=(8,6))
plt.scatter(x,normal_lcd,color='r',zorder=2)
plt.plot(x,normal_lcd,color='r',zorder=1, label= 'Normal')

plt.scatter(x,glaucoma_lcd,color='b',zorder=2)
plt.plot(x,glaucoma_lcd,color='b',zorder=1, label = 'Glaucoma')

plt.xlabel("Plane number", fontsize = 25)
plt.ylabel("LCD (µm)", fontsize = 25)
plt.xticks(fontsize = 18)
plt.yticks(fontsize = 18)

plt.ylim(200, 800)
plt.legend(fontsize='x-large')

plt.savefig('./case_study/lcd_plot.png')
plt.show()